<a href="https://colab.research.google.com/github/NilouGhazavi/Photoreceptor-inspired-CNN-Model-for-Object-Detection/blob/main/PRModel_with_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Functions**


In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import random as random
import matplotlib.animation as animation
import pandas as pd
from random import sample
from tensorflow.keras import Sequential
from tensorflow.python.keras.utils.data_utils import Sequence
from tensorflow.keras.layers import Input, InputLayer

from tensorflow.keras import backend as K
from tensorflow.keras import Model, regularizers, Sequential
from tensorflow.keras.layers import Conv2D, Conv3D, Dense, Activation, Flatten, Reshape, MaxPool3D, MaxPool2D, Permute, BatchNormalization, GaussianNoise,DepthwiseConv2D, Dropout, LayerNormalization
from tensorflow.keras.regularizers import l1, l2
import math
import sys



digit_max=10






#Dataset generating functions (Ines)


def generate_frames_dataset(digits_set, labels_set, n_digits=5, n_frames=1000, 
                            upsample=True, frame_size=280, 
                            downsample=False, pool_size=7, strides=7, final_size=4, 
                            movie=False, duration=5, 
                            perturbations = False, perturbations_time="random", perturbations_duration=2, half=False,
                            spot_proba = 1, spot_factor=100, shadow_factor=0.5,
                           digit_max = 10):
    
    """Generates a dataset of frames with n_digits, with variable frame_size, and possibilitiy to downsample, or movies (with identical frames) if movie=True. Adds light perturbations if perturbations=True, that can be 2 frames long homogenous or half frame light or shadow, at a specific time or random time. 
    
    Args:
    digits_set: np.array of MNIST digits
    labels_set: np.array of int type corresponding to the labels
    n_digits: int
    n_frames: int
    upsample; bool, True or False
    frame_size: if upsample, size of the frame after the upsampling, before the downsampling
    downsample: bool, True or False
    pool_size: int, if downsample
    strides: int, if downsample
    final_size: int
    movie: bool, True or False
    duration: int, if movie
    perturbations: bool, whether to add spotlights and shadows
    perturbations_time: str or int, when to start the 2-frames long perturbations
    half: bool, whether to add the perturbations on only half of the frame (if False : homogeneous perturbation)
    spot_proba: float, between 0 and 1, probability that the perturbation is a spotlight (0.5 if you want equiprobable spotlights and shadows)
    spot_factor: float, by how much to multiply the light intensity
    shadow_factor: float, but how much to multiply the light intensity as well ( < 1 )
    digit_max: maximum of the classes labels, by default 10. Can be less, if you only want to work with some of the classes, and reduce the MNIST dataset you use.
   
    Returns:
    dataset: np.array of frames (n_frames, final_size, final_size) or movies (n_frames, final_size, final_size, duration)
    labels_set: np.array of set of digit_max frames, corresponding to the labels for each class (n_frames, final_size, final_size, digit_max+1)
    
    """
    
    
    digit_shape = np.shape(digits_set[0])
    
    digit_max = len(list(set(labels_set)))
    frames = []
    labels = []
    	
    	
    if upsample:
        
        for i in range(n_frames):
        
            frame = np.zeros((frame_size, frame_size), dtype="float32")
            label = [np.zeros((frame_size, frame_size), dtype="float32" ) for i in range(digit_max)]  #build empty frames and labels


            indexes = random.sample(range(len(digits_set)), n_digits)  #select the MNIST digits, and get the corresponding labels
            digits = digits_set[indexes]
            digits_labels = labels_set[indexes]

            positions = [random.sample(range(frame_size-digit_shape[0]-1),2) for i in range(n_digits)]  #choose the position of the digits
            background = np.ones((frame_size, frame_size), dtype="float32")

            if downsample :
                
                new_frame_size, new_digits, new_positions = little_downsample(frame_size, digits, positions, pool_size, strides)
                #computing the downsampled positions and digits
                frame, label = create_frame_and_labels(new_frame_size, new_digits, digits_labels, new_positions, digit_max)
            
            else:
                
                frame, label = create_frame_and_labels(frame_size, digits, digits_labels, positions, digit_max)
            
            if movie:
            
            	frame = [frame for i in range(duration)]  #making a movie out of identical frames
            	
            	if perturbations:  #adding spotlights and shadows
            	
            		frame = add_perturbations(frame, half, spot_proba, spot_factor, shadow_factor, perturbations_time, movie_duration, final_size)

            labels.append(label)
            frames.append(frame)
        
    else:  #if not upsampled
    
        for i in range(n_frames):

            n = random.randint(0,len(digits_set)-1)

            digit_label = labels_set[n]
            
            frame = digits_set[n]
            label = label_mnist(frame, digit_label, digit_max)

            
            if downsample:
                
                new_frame_size, new_digits, new_positions = little_downsample(frame_size, [frame], [[0,0]], pool_size, strides)
                frame, label = create_frame_and_labels(new_frame_size, new_digits, [digit_label], new_positions, digit_max)

            
            if movie:
            
            	frame = [frame for i in range(duration)]

            	if perturbations:
            	
            		frame = add_perturbations(frame, half, spot_proba, spot_factor, shadow_factor, perturbations_time, perturbations_duration, duration)
				
            labels.append(label)
            frames.append(frame)
            
    dataset = np.array(frames)
    labels_set = np.array(labels)

    #getting the dimensions right for the model
    if not movie:
        dataset = dataset[:,:,:,None]
        labels_set = np.transpose(labels_set, (0,2,3,1))
    

    dataset = np.transpose(dataset, (0,2,3,1))
    labels_set = np.transpose(labels_set, (0,2,3,1))
    
    return dataset, labels_set
 
 
 
 
 
 
 
 
 
 
 


def generate_movie_dataset_5(digits_set, labels_set, 
                            n_movies=1000, duration=5,
                            frame_size = 280, n_digits=5,
                            depth=2, level=1,
                            shadow = False, shadow_ratio = 0.2, light_intensity = 0.1,
                            max_jump=1, speed=1,
                            digit_max = 10
                            ):
    
    """Generates a dataset of movies with moving digits
    
    Args:
    digits_set: np.array of MNIST digits
    labels_set: np.array of the corresponding labels
    n_movies: int, number of movies
    duration: int, duration of the movie
    frame_size: int, size of the frame
    n_digits: int, numer of digits
    shadow: bool, True or False
    shadow_ratio: float, usually 1/10**n, shadow surface/total surface, defines the radius. Reasonable circle size with ratio=0.3
    light_intensity: float, usually 10**n, factor applied to pixels in the shadow
    
    Returns:
    movies: np.array of movies, shape (n_movies, frame_size, frame_size, duration)
    labels: np. array of frames labelling the last frame of each movie for each class
    
    """
    
    radius = int(frame_size*np.sqrt(shadow_ratio/np.pi))
    
    movies, labels = [], []
    
    label_size = frame_size/2**(depth-level-1)

    digit_size = len(digits_set[0][0])
    
    for n in range(n_movies):
        
        # select the digits
        
        indexes = random.sample(range(len(digits_set)), n_digits)
        
        digits = digits_set[indexes]
        digits_labels = labels_set[indexes]
        
        #Initial values for the positions and directions of the digits
		
        positions = [random.sample(range(speed*max_jump, frame_size-digit_size-speed*max_jump),2) for i in range(n_digits)]
        directions =  [non_null_sample(max_jump, 2) for i in range(n_digits)]

        #create the movie and the movie labels
        
        movie = []
        
        for time in range(duration):

            for k, digit in enumerate(digits):
                
                #Compute the new position
                
                position = new_position(positions[k], directions[k], speed)
                positions[k] = position
                
                #Compute the next position to check it's going to be fine
                
                next_position = new_position(positions[k], directions[k], speed)

                while out_of_frame(next_position, frame_size, digit_size):
                
                    #Correct the direction if next position is out of frame
                    
                    rebound_point = closest_frontier(next_position, frame_size, digit_size)
                    directions[k] = mirror_direction(directions[k], rebound_point, frame_size)
                    next_position = new_position(positions[k], directions[k], speed)

                if in_the_center(positions[k], frame_size, speed, digit_size, max_jump):
                
                    #Randomly change the direction, if far enough from the walls
                    directions[k] = change_direction(directions[k], max_jump)
                    
            #Create the frame with the positions of the digits       
            frame = create_frame(frame_size, digits, positions)
            

            if shadow:
                frame = add_shadow(frame, radius, light_intensity)
                
            movie.append(frame)
            
            if time == duration-1 :
            
                    #only label the last time step
            
                    movie_labels = create_labels(frame_size, digits, digits_labels, positions, digit_max)

        movies.append(movie)
        labels.append(movie_labels)
        
    #Get the right shape for the model
    movies = np.transpose(movies, (0,2,3,1))
    labels = np.transpose(labels, (0,2,3,1))
        
    return np.array(movies), np.array(labels)









#Auxiliary functions for the generating functions (Ines)















def add_perturbations(frame, half, spot_proba, spot_factor, shadow_factor, perturbations_time, perturbations_duration, movie_duration):

	"""
	
	Adds light level variations to a frame movie. Either on all the frame, or only half, either spotlight or shadow or both with a probability.
	
	Args:
	frame: 28x28 np array
	half: boolean
	spot_proba: 0 if only shadows, 1 if only spotlights, 0.5 if randomly one or the other.
	spot_factor: factor to multiply the pixel values if spotlight, usually 10**n
	shadow_factor: factor to multiply pixel values if shadow, usually 1/10**n
	perturbations_time: "random" by default, or int, has to be <= movie_duration-perturbations_duration
	perturbations_duration: int, in number of frames
	movie_duration: int
	final_size: int, size of the frame
	
	Returns:
	frame: modified frame
	
	"""
	frame_size = len(frame)

	if perturbations_time=="random":
	
		start_time = random.randint(0,movie_duration-perturbations_duration)
		
	else:
		start_time = perturbations_time
		
	end_time = start_time + perturbations_duration -1
	
	random_nb = np.random.random() #to decide if spotlight or shadow


	if half:
		random_half = np.random.randint(0,2)

		if random_half==1: #low half perturbated
		
			new_frames = []
			for k in range(perturbations_duration):
			
				new_frame = np.vstack([frame[start_time+k][:int(frame_size/2)] * ( (random_nb < spot_proba)*spot_factor + (random_nb >= spot_proba)*shadow_factor ) , frame[start_time+k][int(frame_size/2):]])
				new_frames.append(new_frame)
				
		else: #top half perturbated
		
			new_frames = []
			for k in range(perturbations_duration):
			
				new_frame = np.vstack([frame[start_time+k][:int(frame_size/2)], frame[start_time+k][int(frame_size/2):] * ( (random_nb < spot_proba)*spot_factor + (random_nb >= spot_proba)*shadow_factor )])
				new_frames.append(new_frame)

    			
	else:  #homogeneous perturbation
	
		new_frames = []
		for k in range(perturbations_duration):
		
			new_frame = frame[start_time+k] * ( (random_nb < spot_proba)*spot_factor + (random_nb >= spot_proba)*shadow_factor )
			new_frames.append(new_frame)

	for k in range(perturbations_duration):
	
		frame[start_time+k] = new_frames[k]

	return frame













def add_shadow(frame, radius, intensity):
    
    """Adds a shadow on a frame.
    
    Args:
    frame: np.array
    radius: float
    intensity: factor applied to the pixels in the shadow
    
    Returns:
    shad_frame: np.array of the frame with the shadow
    
    """
    
    frame_size = len(frame)
    shad_frame = frame.copy()
    
    for i,line in enumerate(frame):
        for j,pixel in enumerate(line):
            
            x,y  = i - frame_size/2, j - frame_size/2
            
            if np.sqrt(x**2 + y**2) <= radius :
                shad_frame[i,j] = intensity*frame[i,j]
                
    return shad_frame
















def create_labels(frame_size, digits, digits_labels, positions, digit_max):


    """Create the labels corresponding to a frame.
    
    Args:
    frame_size: float
    digits: MNIST digits present on the frame
    digits_labels: labels of the digits
    positions: 2-uple (x,y) for each digit
    digit_max: 10
    
    Returns:
    labels:frames corresponding to each class
    
    """
    
    labels = np.array([np.zeros((frame_size, frame_size)) for i in range(digit_max)]+ [np.ones((frame_size, frame_size))])

    for k, digit in enumerate(digits):

        
        frame = labels[digits_labels[k]]



        label_k = insert(digit, frame, positions[k])


        labels[digits_labels[k]] = label_k


        labels[-1] = np.maximum(labels[-1]-labels[digits_labels[k]], 0)
        
    return labels

















def label_mnist(digit_frame, digit, digit_max=10):

	
	"""
	
	Creates the labels frames corresponding to an MNIST digit
	
	Args:
	digit_frame: 28x28 np array, MNIST digit directly
	digit: int, MNIST label
	digit_max: int, default to 10
	
	Returns:
	labels: digit_max+1 frames, shape (digit_max+1, 28,28), with zeros on every frame but the digit frame, where the non zero MNIST pixels are =1.
	
	"""
	
	labels = [np.zeros(np.shape(digit_frame), dtype="float32") for i in range(digit_max+1)]

	label = np.zeros(np.shape(digit_frame), dtype="float32")
	for i in range(len(digit_frame)):
		for j in range(len(digit_frame[i])):
	    		if digit_frame[i][j] != 0:
	    			label[i][j] = 1

	labels[digit] = label
	labels[-1] = 1-label

	return labels















def insert(digit, frame, position):
    
    """
    
    Adds a digit in the frame. If there is already a digit, it has priority (the other is "hidden" behind)
    
    Args:
    digit: 28x28 array
    frame: array
    position: x,y for the digit's top left corner
    
    Outputs:
    frame: array
    
    """

    max_value = np.max(digit)
    for i, line in enumerate(digit):

        for j, pixel in enumerate(line):

            frame_value = frame[position[0]+i, position[1]+j]
            
            if frame_value == 0 :
                frame[position[0]+i, position[1]+j] = pixel
                
            #else : already a prioritary digit here

    
    return frame















def create_frame_and_labels(frame_size, digits, digits_labels, positions, digit_max):
    
    """Create a frame and the corresponding labels.
    
    Args:
    frame_size: int
    digits: np.array of MNIST digits to put on the frame
    digits_labels: np.array of int labels corresponding to the digits
    positions: 2-uple (x,y) for each digit
    digit_max: 10
    
    Returns:
    frame: the created frame
    labels: the associated label frames for each class
    
    """
    
    frame = np.zeros((frame_size, frame_size))
    background = np.ones((frame_size, frame_size))
    labels = [np.zeros((frame_size, frame_size)) for i in range(digit_max)]
    
    for k, digit in enumerate(digits):
        
        digit_label = digits_labels[k]
        position = positions[k]
        
        frame, labels[int(digit_label)], background = insert_and_label(digit, digit_label, frame, labels[int(digit_label)], position, background)
        
    labels.append(background)
        
    return np.array(frame), np.array(labels)

















def insert_and_label(digit, digit_label, frame, proba, position, background):

    """
    
    Adds a digit to a frame and to the corresponding labels
    
    Args:
    digit: MNIST digit
    digit_label: int
    frame: frame to insert the digit in
    proba: label drame corresponding to the digit
    position: (x,y) digit position
    background: background frame in the labels
    
    Returns:
    frame: modified frame
    new_proba: modified label frame
    background: modified background frame
    
    """
    

    new_proba = proba.copy()
    max_value = np.max(digit)
    
    for i, line in enumerate(digit):
        for j, pixel in enumerate(line):
            
            frame_value = frame[position[0]+i, position[1]+j]
            
            if frame_value == 0:
                frame[position[0]+i, position[1]+j] = pixel
                
            #else : already a digit there, priority to the first arrived
                
            if pixel != 0:
                
                new_proba[position[0]+i][position[1]+j] = 1
                background[position[0]+i][position[1]+j] = 0
                #where a non-zero digit pixel is inserted, a one is inserted in the same position of the corresponding matrix


    
    return frame, new_proba, background

















def downsample_digit(digits_list, 
                     pool_size, strides
                    ):
                    
    """
    
    Downsamples digits
    
    Args:
    digits_list: list of MNIST digits
    pool_size: int, for the downsampling
    strides: int, for the downsampling
    
    Returns:
    down_digits: list of downsampled digits
    new_digit_size: int
    
    """
    
    down_digits = []
    
    digit_size = len(digits_list[0][0])

    
    for digit in digits_list :


        down_digit = []
        line = 0

        while line < len(digit[0])-pool_size+1 :
            new_line = []
            column = 0
            while column < len(digit[0])-pool_size+1 :
        
                new_line.append(np.max( digit[line:line+pool_size, column:column+pool_size]))
                column += strides
        
            line += strides
            down_digit.append(new_line)
        
        down_digit= np.array(down_digit)


        down_digits.append(down_digit)
        new_digit_size = len(down_digits[0])
        
    return down_digits, new_digit_size












def little_downsample(frame_size, digits, positions, pool_size, strides):
    
    """Computes the new frame size, the new MNIST digits and their new positions after the downsampling. 
    Allows to create the downsampled frame rather then downsampling a frame, faster.
    
    Args:
    frame_size; int, before downsampling
    digits: MNIST digits
    positions: 2-uple (x,y) for each digit
    pool_size: int
    strides: int
    
    Returns:
    down_frame_size: int, new frame size
    down_digits: np.array of new digits
    down_positions: 2-uple of new (x,y) position for each digit
    
    """
    
    digit_size = len(digits[0][0])
    
    down_frame_size = (frame_size-pool_size)/strides +1
    down_positions = []
    down_digits = []
    
    for position in positions:
        x,y = position
        down_x, down_y = x//strides, y//strides
        down_positions.append([down_x,down_y])
        
        
    down_digits, digit_size = downsample_digit(digits, pool_size, strides)

        
    return int(down_frame_size), down_digits, down_positions
















def create_frame(frame_size, digits, positions):
    
    """
    
    Creates the frame zith all the digits.
    
    Args:
    frame_size: for a square frame
    digits: list of 28x28 arrays
    positions: list of x,y corresponding to the digits' top left corners, in the same order
    
    Outputs:
    frame: 280x280 array
    
    """
    
    frame = np.zeros((frame_size, frame_size))
    for k, digit in enumerate(digits):
        position = positions[k]
        frame = insert(digit, frame, position)
    return frame

















def mirror_direction(direction, position, frame_size):
    
    """
    
    Finds the new direction when a digit rebounds on a wall.
    
    Args:
    direction: current direction of the digit [dx,dy] 
    position: current position of the digit [x,y]
    frame_size: for a square frame
    
    Outputs:
    direction: [dx,dy]
    
    """
    
    dx, dy = direction
    x,y = position
    if (x==0 or x==frame_size-1) and y!=0 and y!=frame_size-1:
        dx = -dx
    if (y==0 or y==frame_size-1) and x!=0 and x!=frame_size-1:
        dy = -dy
    if (x==0 or x==frame_size-1) and (y==0 or y==frame_size-1):
        dx, dy = -dx, -dy
    return [dx, dy]


















def change_direction(direction, max_jump):
    
    """
    
    Generates a slight change in direction, called with a small probability.
    
    Args:
    direction: current direction [dx,dy]
    max_jump: initially defined constant
    
    Outputs:
    direction: new direction[dx,dy]
    
    """
    
    random_nb = random.random()
    if random_nb < 0.05:
        direction[0] = min(max_jump, max(-max_jump,direction[0]+1))
        # to check that still between -max_jump and max_jump
    elif random_nb >= 0.05 and random_nb < 0.1 :
        direction[0] = min(max_jump, max(-max_jump,direction[0]-1))
    if random_nb >= 0.1 and random_nb < 0.15:
        direction[1] = min(max_jump, max(-max_jump,direction[1]+1))
    elif random_nb >= 0.15 and random_nb < 0.2 :
        direction[1] = min(max_jump, max(-max_jump,direction[1]-1))
    return direction



















def closest_frontier(position, frame_size, digit_size):

	"""
	
	Finds the closest point on the walls to a digit
	
	Args:
	position: (x,y) position of the top left corner of the digit
	frame_size: int
	digit_size: int
	
	Returns:
	(x,y) position of the closest point on a wall
	
	"""
    
	x,y = position
	neighbor_frontiers = [(frame_size-1,y), (0,y), (x, frame_size-1), (x,0)]
	distances = []
	distances.append(frame_size - (x+digit_size))
	distances.append(x)
	distances.append(frame_size - (y+digit_size))
	distances.append(y)
	return neighbor_frontiers[distances.index(min(distances))]



















def non_null_sample(max_value, number):

	"""
	
	Selects a number-uple of values different from (0,...,0) for the directions, usually (dx,dy) because 2D
	
	Args:
	max_value: int, max value to choose
	number: number of numbers to pick
	
	Returns:
	number-uple of values not all zero, usually (dx,dy)
	
	"""
    
	values = list(np.arange(-max_value, max_value+1))
	sample = []
	for i in range(number-1):
		sample.append(random.choice(values))
	#for the last value, we cannot add a zero if there are only zero in the selected values
	if list(set(sample)) != [0]:
		sample.append(random.choice(values))
	else:
		values.remove(0)
		sample.append(random.choice(values))
	return sample

















def new_position(position, direction, speed):

	"""
	
	Computes the new position from current position, direction and speed
	
	Args:
	position: (x,y) int 
	direction: (dx, dy) int
	speed: int
	
	Returns:
	(x,y) new position : x += speed*dx
	
	"""
	
	x,y = position
	dx, dy = direction
	return [x+speed*dx, y+speed*dy]

















def out_of_frame(position, frame_size, digit_size):

	"""
	
	Checks whether a position would go out of the frame
	
	Args:
	position: (x,y) int
	frame_size: int
	digit_size: int
	
	Returns:
	bool: True if out of frame
	
	"""
	for i, dimension in enumerate(position) :

		if dimension + digit_size  > frame_size:
			return True
		if dimension < 0:
			return True
	return False


















def in_the_center(position, frame_size, speed, digit_size, max_jump):
	"""
	
	Checks whether a digit is far enough from the walls (to randomly change direction without danger)
	
	Args:
	position: (x,y) int
	frame_size: int
	speed: int
	digit_size: int
	max_jump: int
	
	Returns:
	bool: True if far enough from the walls
	
	"""
	
	security_distance = abs(speed*max_jump)+ digit_size
	for dimension in position :
		if abs(frame_size-dimension)<security_distance or dimension < security_distance:
	    		return False
	return True


















def save_movie(movie, title, directory, time_interval=70):
    
    
    """
    Saves a movie in a directory in GIF format
    
    Args:
    movie: np.array of dimension (n_frames, frame_size, frame_size)
    title: string
    directory: str of shape "folder1/folder2/"
    time_interval: int, between two frames. 50 is fast, 150 is slow
    
    Returns:
    nothing, the movie is saved.
    
    """
    
    fig, ax  = plt.subplots()
    ims = []
    for i, image in enumerate(movie):
        im = ax.imshow((image), animated=True)
        if i == 0:
            ax.imshow(image) 
        ims.append([im])

    gif = animation.ArtistAnimation(fig, ims, interval=time_interval, blit=True,
                                repeat_delay=1000)
    gif.save(directory+title+".gif")
    
    
    
    
    
    
    
    
    
#Custom metrics (Ines)














@tf.function(experimental_relax_shapes=True)
def new_weighted_loss_0(y_true, y_pred):

    """
    Counts the number of zeros and ones pixels on the labels, then applies the weights on all the frames based on the background to balance the digits and background classes (because a lot more zeros). Then computes the weighted cross-entropy loss (- mean of true_labels * weights * log(predictions) )
    
    Args:
    y_true: true labels
    y_pred: predictions
    
    Returns:
    Cross-entropy loss
    
    """

    background = y_true[:, :,-1]
    w1 = tf.reduce_sum(background)/tf.cast(tf.size(background), tf.float32)
    w2 = 1-w1
    w = background*w2 + (1-background)*w1
    
    #loss = tf.reduce_mean(w[:, :, None]*tf.square(tf.subtract(y_true, y_pred))) 
    #to broadcast (over channels)
    #tf.print(tf.math.log(y_pred))

    y_pred = tf.clip_by_value(y_pred, 0.0001, 0.9999)
    term = w[:,:, None]* (y_true)* tf.math.log(y_pred) 
    loss =tf.reduce_mean(term)
    #tf.print(loss)

    return tf.math.abs(loss)
    
    
    
    
    
    
    
    
    
    
    
    
    
@tf.function
def new_weighted_loss(y_true, y_pred):

    """
    Counts the number of zeros and ones pixels on the labels, then applies the weights on each frame separately to balance the zeros and ones classes (because a lot more zeros). Then computes the weighted cross-entropy loss (- mean of true_labels * weights * log(predictions) )
    
    Args:
    y_true: true labels
    y_pred: predictions
    
    Returns:
    Cross-entropy loss
    
    """

    w1 = tf.reduce_sum(y_true)/tf.cast(tf.size(y_true), tf.float32)  #proportion of ones pixels
    w2 = 1-w1   #proportion of zeros pixels
    
    N = len(y_true[0,0,0])  # number of classes
    weights = tf.TensorArray(tf.float32, size=N)
    
    #now we build the weights matrix
    
    for i in range(N):
    
    	frame = y_true[:,:,:,i]
    	weights_frame = frame*w2 + (1-frame)*w1   #ones pixels get the zeros pixels proportion as their weight, for each frame
    	
    	weights = weights.write(i, weights_frame)
    	
    weights = tf.transpose(weights.stack(), (1, 2, 3, 0))
        
    y_pred = tf.clip_by_value(y_pred, 0.0001, 0.9999)
    #to avoid log(0) that was causing an error
    term = weights * (y_true)* tf.math.log(y_pred) 
    loss = - tf.reduce_mean(term)

    return loss















@tf.function(experimental_relax_shapes=True)
def my_accuracy(y_true, y_pred):

	"""
	Computes the mean of exp(-abs(relative_error)), only taking into account digit pixels, and not the background. Idea: to get a better range of the accuracy values, we could add a multiplying factor inside the exp.
	
	Args:
	y_true: true labels
	y_pred: predictions
	
	Returns:
	non-categorical accuracy
	
	"""
	
	y_true_digits = y_true[:,:,:,:-1]
	y_pred_digits = y_pred[:,:,:,:-1]
	background = y_true[:,:,:,:-1]
	n_classes = y_true.get_shape()[-1]-1
	
	#What I used in my runs
	similarity = tf.math.exp(- tf.math.abs( y_pred_digits - y_true_digits) )
	
	#What I realized on the presentation day would have been better
	#1. Using the relative error
	#2. Using a factor to decide how low the accuracy can go (right now, betwen 0.9 and 1...)
	#similarity = tf.math.exp(- tf.math.abs( factor * (y_pred_digits - y_true_digits)/y_true_digits )
	


	digit_index = tf.math.argmax(tf.math.reduce_max(y_true[:,:,:,:-1], axis=(0,1,2))) 
	digit_frame = y_true_digits[:,:,:,digit_index]	
	
	weighted_similarity = digit_frame[:,:,:,None] * similarity



	my_accuracy = tf.cast(tf.reduce_sum( weighted_similarity ), tf.float32) / tf.cast(tf.math.count_nonzero(digit_frame) * n_classes, tf.float32)
	
	return my_accuracy

	
	
	
	
	
	
	
	
	
	
	
	
@tf.function(experimental_relax_shapes=True)
def my_cat_accuracy(y_true, y_pred):

	"""
	Computes the frequency of good predictions, choosing the maximum probability over the classes, only taking into account digit pixels, and not the background.
	
	Args:
	y_true: true labels
	y_pred: predictions
	
	Returns:
	categorical accuracy
	
	"""

	background = y_true[:,:,:,-1]
	
	#we "force the network to choose" by taking the maximum over the classes
	indexes_true = tf.math.argmax(y_true,axis=3)
	indexes_pred = tf.math.argmax(y_pred,axis=3)
	
	#the "indexes" are just one frame wih values between 0 and 10, indicating the class with the maximum probability for each pixel

	#find where the digit actually is
	digit_index = tf.math.argmax(tf.math.reduce_max(y_true[:,:,:,:-1], axis=(0,1,2))) 
	
	#the reduce_max is list of n_classes element, excluding the background. the max is the frame with the actual digit (the others are only zeros)

	#we count the pixels where the predicted class is correct, and is the digit (not background) 
	acc = tf.math.reduce_sum(tf.cast((indexes_true == indexes_pred) & (indexes_true==digit_index), tf.float32)) / tf.cast(tf.math.count_nonzero(1-background), tf.float32)

	return acc









#Code for the photoreceptor model (Saad)


def conv_oper_multichan(x,kernel_1D):

    spatial_dims = x.shape[-1]
    x_reshaped = tf.expand_dims(x,axis=2)
    kernel_1D = tf.squeeze(kernel_1D,axis=0)
    kernel_1D = tf.reverse(kernel_1D,[0])
    tile_fac = tf.constant([spatial_dims,1])
    kernel_reshaped = tf.tile(kernel_1D,(tile_fac))
    kernel_reshaped = tf.reshape(kernel_reshaped,(1,spatial_dims,kernel_1D.shape[0],kernel_1D.shape[-1]))
    kernel_reshaped = tf.experimental.numpy.moveaxis(kernel_reshaped,-2,0)
    pad_vec = [[0,0],[kernel_1D.shape[0]-1,0],[0,0],[0,0]]
    # # pad_vec = [[0,0],[0,0],[0,0],[0,0]]
    # conv_output = tf.nn.conv2d(x_reshaped,kernel_reshaped,strides=[1,1,1,1],padding=pad_vec,data_format='NHWC')
    conv_output = tf.nn.depthwise_conv2d(x_reshaped,kernel_reshaped,strides=[1,1,1,1],padding=pad_vec,data_format='NHWC')
    
    # print(conv_output.shape)
    return conv_output




@tf.function()#experimental_relax_shapes=True)
def slice_tensor(inp_tensor,shift_vals):
    # print(inp_tensor.shape)
    # print(shift_vals.shape)
    shift_vals = shift_vals[:,tf.newaxis,:]
    shift_vals = tf.tile(shift_vals,[1,inp_tensor.shape[-2],1])
    tens_reshape = tf.reshape(inp_tensor,[-1,inp_tensor.shape[1]*inp_tensor.shape[2]*inp_tensor.shape[3]*inp_tensor.shape[4]])
    # print('tens_reshape: ',tens_reshape.shape)
    shift_vals_new = (inp_tensor.shape[1]-shift_vals[0])*(inp_tensor.shape[-1]*inp_tensor.shape[-2])
    rgb = tf.range(0,shift_vals_new.shape[-1])
    rgb = rgb[tf.newaxis,:]
    rgb = tf.tile(rgb,[shift_vals_new.shape[0],1])
    temp = tf.range(0,shift_vals_new.shape[-1]*shift_vals_new.shape[-2],shift_vals_new.shape[-1])
    rgb = rgb+temp[:,None]
    shift_vals_new = shift_vals_new + rgb
    extracted_vals = tf.gather(tens_reshape,shift_vals_new,axis=1)
    # print('extracted_vals: ',shift_vals.shape)
    extracted_vals_reshaped = tf.reshape(extracted_vals,(-1,1,inp_tensor.shape[2],inp_tensor.shape[3],inp_tensor.shape[4]))
    
    return extracted_vals_reshaped





def generate_simple_filter_multichan(tau,n,t):
    t_shape = t.shape[0]
    t = tf.tile(t,tf.constant([tau.shape[-1]], tf.int32))
    t = tf.reshape(t,(tau.shape[-1],t_shape))
    t = tf.transpose(t)
    f = (t**n[:,None])*tf.math.exp(-t/tau[:,None]) # functional form in paper
    rgb = tau**(n+1)
    f = (f/rgb[:,None])/tf.math.exp(tf.math.lgamma(n+1))[:,None] # normalize appropriately
    # print(t.shape)
    # print(n.shape)
    # print(tau.shape)
   
    return f
    
class photoreceptor_DA_multichan_randinit(tf.keras.layers.Layer):
    def __init__(self,units=1,kernel_regularizer=None):
        super(photoreceptor_DA_multichan_randinit,self).__init__()
        self.units = units
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        
    def get_config(self):
         config = super().get_config()
         config.update({
             "units": self.units,
             "kernel_regularizer": self.kernel_regularizer
         })
         return config      
               
    def build(self,input_shape):    # random inits


        # # # # Constant Values
        # zeta_init = tf.keras.initializers.Constant(0.2) 
        # self.zeta = self.add_weight(name='zeta',initializer=zeta_init,shape=[1,self.units],trainable=False)
        # zeta_mulFac = tf.keras.initializers.Constant(1000.) 
        # self.zeta_mulFac = self.add_weight(name='zeta_mulFac',initializer=zeta_mulFac,shape=[1,self.units],trainable=False)
    
        # # # you can have a costant value constant value for kappa
        # kappa_init = tf.keras.initializers.Constant(0.13) 
        # self.kappa = self.add_weight(name='kappa',initializer=kappa_init,shape=[1,self.units],trainable=False)
        # kappa_mulFac = tf.keras.initializers.Constant(1000.) 
        # self.kappa_mulFac = self.add_weight(name='kappa_mulFac',initializer=kappa_mulFac,shape=[1,self.units],trainable=False)

        # # # # you can have a constant value for alpha 
        # alpha_init = tf.keras.initializers.Constant(0.0667) 
        # self.alpha = self.add_weight(name='alpha',initializer=alpha_init,shape=[1,self.units],trainable=False)
        # alpha_mulFac = tf.keras.initializers.Constant(1000.) 
        # self.alpha_mulFac = self.add_weight(name='alpha_mulFac',initializer=alpha_mulFac,shape=[1,self.units],trainable=False)

        # # # # you can have a constant value for beta 
        # beta_init = tf.keras.initializers.Constant(0) 
        # self.beta = self.add_weight(name='beta',initializer=beta_init,shape=[1,self.units],trainable=False)
        # beta_mulFac = tf.keras.initializers.Constant(1000.) 
        # self.beta_mulFac = self.add_weight(name='beta_mulFac',initializer=beta_mulFac,shape=[1,self.units],trainable=False)

        # # # you can have a costant value constant value for gamma
        # gamma_init = tf.keras.initializers.Constant(0.1) 
        # self.gamma = self.add_weight(name='gamma',initializer=gamma_init,shape=[1,self.units],trainable=False)
        # gamma_mulFac = tf.keras.initializers.Constant(10.) 
        # self.gamma_mulFac = self.add_weight(name='gamma_mulFac',initializer=gamma_mulFac,shape=[1,self.units],trainable=False)


        # # # you can have a costant value constant value for tauY
        # tauY_init = tf.keras.initializers.Constant(0.06) 
        # self.tauY = self.add_weight(name='tauY',initializer=tauY_init,shape=[1,self.units],trainable=False)
        # tauY_mulFac = tf.keras.initializers.Constant(10.) 
        # self.tauY_mulFac = self.add_weight(name='tauY_mulFac',initializer=tauY_mulFac,shape=[1,self.units],trainable=False)

        # # # you can have a costant value constant value for nY
        # nY_init = tf.keras.initializers.Constant(0.1) 
        # self.nY = self.add_weight(name='nY',initializer=nY_init,shape=[1,self.units],trainable=False)
        # nY_mulFac = tf.keras.initializers.Constant(10.) 
        # self.nY_mulFac = self.add_weight(name='nY_mulFac',initializer=nY_mulFac,shape=[1,self.units],trainable=False)

        # # # you can have a costant value constant value for tauz
        # tauZ_init = tf.keras.initializers.Constant(0.09) 
        # self.tauZ = self.add_weight(name='tauZ',initializer=tauZ_init,shape=[1,self.units],trainable=False)
        # tauZ_mulFac = tf.keras.initializers.Constant(100.) 
        # self.tauZ_mulFac = self.add_weight(name='tauZ_mulFac',initializer=tauZ_mulFac,shape=[1,self.units],trainable=False)

        # # # you can have a costant value constant value for nz
        # nZ_init = tf.keras.initializers.Constant(0.09) 
        # self.nZ = self.add_weight(name='nZ',initializer=nZ_init,shape=[1,self.units],trainable=False)
        # nZ_mulFac = tf.keras.initializers.Constant(10.) 
        # self.nZ_mulFac = self.add_weight(name='nZ_mulFac',initializer=nZ_mulFac,shape=[1,self.units],trainable=False)




# Range of values for each parameter 


    
## 
        
        zeta_range = (0,1)
        # zeta_range = (.02602935,.02602935000001)
        # zeta_range = (.02,.02000001)
        zeta_init = tf.keras.initializers.RandomUniform(minval=zeta_range[0],maxval=zeta_range[1]) #tf.keras.initializers.Constant(0.0159) 
        self.zeta = self.add_weight(name='zeta',initializer=zeta_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        zeta_mulFac = tf.keras.initializers.Constant(1000.) 
        self.zeta_mulFac = self.add_weight(name='zeta_mulFac',initializer=zeta_mulFac,shape=[1,self.units],trainable=False)
        


        # # you can have a range for kappa 
        kappa_range = (0,1)
        # kappa_range = (0.05,0.0500001)
        # kappa_range = (0.049,0.049000001)
        kappa_init = tf.keras.initializers.RandomUniform(minval=kappa_range[0],maxval=kappa_range[1]) #tf.keras.initializers.Constant(0.0159) 
        self.kappa = self.add_weight(name='kappa',initializer=kappa_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        kappa_mulFac = tf.keras.initializers.Constant(1000.) 
        self.kappa_mulFac = self.add_weight(name='kappa_mulFac',initializer=kappa_mulFac,shape=[1,self.units],trainable=False)
        


        # # you can have a range for alpha 
        alpha_range = (0,1)
        # alpha_range = (0.03487017,0.0348701700001)
        # alpha_range = (0.05,0.05000001)
        alpha_init = tf.keras.initializers.RandomUniform(minval=alpha_range[0],maxval=alpha_range[1]) #tf.keras.initializers.Constant(0.0159) 
        self.alpha = self.add_weight(name='alpha',initializer=alpha_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        alpha_mulFac = tf.keras.initializers.Constant(100.) 
        self.alpha_mulFac = self.add_weight(name='alpha_mulFac',initializer=alpha_mulFac,shape=[1,self.units],trainable=False)
        


        # # you can have a range for beta
        beta_range = (0,1)
        # beta_range = (0.02,0.02000001)
        # beta_range = (0.04995,0.04995000001)
        beta_init = tf.keras.initializers.RandomUniform(minval=beta_range[0],maxval=beta_range[1])  #tf.keras.initializers.Constant(0.02)# 
        self.beta = self.add_weight(name='beta',initializer=beta_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        beta_mulFac = tf.keras.initializers.Constant(10.) 
        self.beta_mulFac = self.add_weight(name='beta_mulFac',initializer=beta_mulFac,shape=[1,self.units],trainable=False)
 
  

        # # you can have a range for gamma
        gamma_range = (0,1)
        # gamma_range = (0.02,0.02000001)
        gamma_init = tf.keras.initializers.RandomUniform(minval=gamma_range[0],maxval=gamma_range[1])  #tf.keras.initializers.Constant(0.075)# 
        self.gamma = self.add_weight(name='gamma',initializer=gamma_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        gamma_mulFac = tf.keras.initializers.Constant(10.) 
        self.gamma_mulFac = self.add_weight(name='gamma_mulFac',initializer=gamma_mulFac,shape=[1,self.units],trainable=False)


        #You can have a rannge for tauy, multiplication factor is 1000 from frame 1 to frame 20 
        tauY_range = (0,1)
        # tauY_range = (0.02,0.0200001)
        # tauY_range = (0.04997,0.04997000001)
        tauY_init = tf.keras.initializers.RandomUniform(minval=tauY_range[0],maxval=tauY_range[1])  #tf.keras.initializers.Constant(0.01)# 
        self.tauY = self.add_weight(name='tauY',initializer=tauY_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        tauY_mulFac = tf.keras.initializers.Constant(100.) #tf.keras.initializers.Constant(100.) 
        self.tauY_mulFac = tf.Variable(name='tauY_mulFac',initial_value=tauY_mulFac(shape=(1,self.units),dtype='float32'),trainable=False)
 

        # you can have a range for nY
        nY_range = (0,1)
        # nY_range = (0.02,0.0200001)
        # nY_range = (0.02,0.02000001)
        nY_init = tf.keras.initializers.RandomUniform(minval=nY_range[0],maxval=nY_range[1]) #tf.keras.initializers.Constant(0.01)# 
        self.nY = self.add_weight(name='nY',initializer=nY_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        nY_mulFac = tf.keras.initializers.Constant(10.) 
        self.nY_mulFac = tf.Variable(name='nY_mulFac',initial_value=nY_mulFac(shape=(1,self.units),dtype='float32'),trainable=False)


        # you can have a range for tauz , multiplication factor is 100 from frame 2 to frame 20 
        tauZ_range = (0,1)
        # tauZ_range = (0.05,0.0500001)
        # tauZ_range = (0.0208,0.0208000001)
        tauZ_init = tf.keras.initializers.RandomUniform(minval=tauZ_range[0],maxval=tauZ_range[1]) #tf.keras.initializers.Constant(0.5)# 
        self.tauZ = self.add_weight(name='tauZ',initializer=tauZ_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        tauZ_mulFac = tf.keras.initializers.Constant(100.) 
        self.tauZ_mulFac = tf.Variable(name='tauZ_mulFac',initial_value=tauZ_mulFac(shape=(1,self.units),dtype='float32'),trainable=False)
                

        # you can have a range for nz
        nZ_range = (0,1)
        # nZ_range = (0.04983,0.04983000001)
        nZ_init = tf.keras.initializers.RandomUniform(minval=nZ_range[0],maxval=nZ_range[1])  #tf.keras.initializers.Constant(0.01)# 
        self.nZ = self.add_weight(name='nZ',initializer=nZ_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        nZ_mulFac = tf.keras.initializers.Constant(10.) 
        self.nZ_mulFac = tf.Variable(name='nZ_mulFac',initial_value=nZ_mulFac(shape=(1,self.units),dtype='float32'),trainable=False)





# remove kc 
        # multiplication factor is 100 from frame 1 to frame 20 
        tauC_range = (0.02,0.05)
        tauC_init = tf.keras.initializers.RandomUniform(minval=tauC_range[0],maxval=tauC_range[1])  #tf.keras.initializers.Constant(0.2)# 
        self.tauC = self.add_weight(name='tauC',initializer=tauC_init,shape=[1,self.units],trainable=True,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,tauC_range[0],tauC_range[1]))
        tauC_mulFac = tf.keras.initializers.Constant(100.) 
        self.tauC_mulFac = tf.Variable(name='tauC_mulFac',initial_value=tauC_mulFac(shape=(1,self.units),dtype='float32'),trainable=False)
                
        nC_range = (1e-5,0.2)
        nC_init = tf.keras.initializers.RandomUniform(minval=nC_range[0],maxval=nC_range[1]) 
        self.nC = self.add_weight(name='nC',initializer=nC_init,shape=[1,self.units],trainable=False,regularizer=self.kernel_regularizer,constraint=lambda x: tf.clip_by_value(x,nC_range[0],nC_range[1]))
        nC_mulFac = tf.keras.initializers.Constant(10.) 
        self.nC_mulFac = tf.Variable(name='nC_mulFac',initial_value=nC_mulFac(shape=(1,self.units),dtype='float32'),trainable=False)
    
    def call(self,inputs):
       
        timeBin = 1
        
        alpha =  self.alpha*self.alpha_mulFac
        beta = self.beta*self.beta_mulFac
        gamma =  self.gamma*self.gamma_mulFac
        zeta = self.zeta*self.zeta_mulFac
        kappa = self.kappa*self.kappa_mulFac
        tau_y =  (self.tauY_mulFac*self.tauY) / timeBin
        tau_z =  (self.tauZ_mulFac*self.tauZ) / timeBin
        tau_c =  (self.tauC_mulFac*self.tauC) / timeBin
        n_y =  (self.nY_mulFac*self.nY)
        n_z =  (self.nZ_mulFac*self.nZ)
        n_c =  (self.nC_mulFac*self.nC)
        
        # print('tau_z: ',tau_y.shape)
        
        t = tf.range(0,inputs.shape[1],dtype='float32')
        
        Ky = generate_simple_filter_multichan(tau_y,n_y,t)   
        Kc = generate_simple_filter_multichan(tau_c,n_c,t)  
        Kz = generate_simple_filter_multichan(tau_z,n_z,t) 
        # Kz = (gamma*Kc) + ((1-gamma) * Kz)
        # replace kc by ky  
        Kz = (gamma*Ky) + ((1-gamma) * Kz)
        # print('Kz: ',Kz.shape)
        
        # Kz = Kz[None,0,:]
        # print('Kz_new',Kz.shape)
        
        # print('inputs: ',inputs.shape)
        y_tf = conv_oper_multichan(inputs,Ky)
        z_tf = conv_oper_multichan(inputs,Kz)
        # print('z_tf: ',z_tf.shape)
               
        y_tf_reshape = tf.reshape(y_tf,(-1,y_tf.shape[1],y_tf.shape[2],inputs.shape[-1],tau_z.shape[-1]))
        z_tf_reshape = tf.reshape(z_tf,(-1,z_tf.shape[1],z_tf.shape[2],inputs.shape[-1],tau_z.shape[-1]))
        # print('z_tf_reshape: ',z_tf_reshape.shape)
        

        ## For time distributed, remove these four lines:
        
        # y_shift = tf.math.argmax(Ky,axis=1);y_shift = tf.cast(y_shift,tf.int32)
        # z_shift = tf.math.argmax(Kz,axis=1);z_shift = tf.cast(z_shift,tf.int32)
        
        # y_tf_reshape = slice_tensor(y_tf_reshape,y_shift)
        # z_tf_reshape = slice_tensor(z_tf_reshape,z_shift)
        # print('z_tf_slice: ',z_tf_reshape.shape)
               
    
        outputs = (zeta[None,None,0,None,:] + (alpha[None,None,0,None,:]*y_tf_reshape[:,:,0,:,:]))/(kappa[None,None,0,None,:]+1e-6+(beta[None,None,0,None,:]*z_tf_reshape[:,:,0,:,:]))       

        # set kappa=1, and zeta=0
        # outputs = (0 + (alpha[None,None,0,None,:]*y_tf_reshape[:,:,0,:,:]))/(1+1e-6+(beta[None,None,0,None,:]*z_tf_reshape[:,:,0,:,:]))       

        # print(outputs.shape)
        
        return outputs




class Normalize_multichan(tf.keras.layers.Layer):
    """
    BatchNorm is where you calculate normalization factors for each dimension seperately based on
    the batch data
    LayerNorm is where you calculate the normalization factors based on channels and dimensions
    Normalize_multichan calculates normalization factors based on all dimensions for each channel seperately
    """
    
    def __init__(self,units=1):
        super(Normalize_multichan,self).__init__()
        self.units = units
        
    def get_config(self):
         config = super().get_config()
         config.update({
             "units": self.units,
         })
         return config   
             
    def call(self,inputs):
        inputs_perChan = tf.reshape(inputs,(-1,inputs.shape[-1]))
        value_min = tf.reduce_min(inputs_perChan,axis=0)
        value_max = tf.reduce_max(inputs_perChan,axis=0)
        
        # value_min = tf.expand_dims(value_min,axis=0)
        R_norm = (inputs - value_min[None,None,None,None,:])/(value_max[None,None,None,None,:]-value_min[None,None,None,None,:])
        R_norm_perChan = tf.reshape(R_norm,(-1,R_norm.shape[-1]))
        R_mean = tf.reduce_mean(R_norm_perChan,axis=0)       
        R_norm = R_norm - R_mean[None,None,None,None,:]
        return R_norm














	
	
	
	
#(Saad)
def get_weightsDict(model):
    names = [weight.name for layer in model.layers for weight in layer.weights]
    weights = model.get_weights()
    weights_dict = {}
    for i in range(len(names)):
        weight_name = names[i][:-2]
        weights_dict[weight_name] = np.atleast_1d(np.squeeze(weights[i]))
    return weights_dict
		









#Block functions for the model (Ines)













def conv_block(x, n_filters):
    """Conv2D then ReLU activation"""
    x = layers.Conv2D(n_filters, 3, 
                      padding = "same", 
                      activation = "relu", 
                      kernel_initializer = "he_normal")(x)

    return x










def double_conv_block(x, n_filters):
    """ 2 Conv2D """
    x = layers.Conv2D(n_filters, 3, 
                      padding = "same", 
                      activation = "relu", 
                      kernel_initializer = "he_normal")(x)

    x = layers.Conv2D(n_filters, 3, 
                      padding = "same", 
                      activation = "relu", 
                      kernel_initializer = "he_normal")(x)
    return x













def downsample_block(x, n_filters):
    """conv_bloc, MaxPool and Dropout"""
    f = conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p














def upsample_block(x, conv_features, n_filters):
    """ Conv2DTranspose, concatenate, Dropout, conv_block"""

    x = layers.Conv2DTranspose(n_filters, 2, 2, padding="same")(x)

    x = layers.concatenate([x, conv_features])

    x = layers.Dropout(0.3)(x)

    x = conv_block(x, n_filters)
    return x









#Functions to build the models (Ines)








    

def build_photo_unet_model(size, duration, photo, digit_max=10, depth =2, level=1, chan1_n = 9):
    
    """
    Builds a U-net model with the photoreceptors model, 
    then a convolutional U-net with "depth" blocks on each side.
    
    Args:
    size: int, frame_size of the input
    duration: int, duration of the movies
    digit_max: int, default to 10
    depth: int, number of blocks in the model on each side, default to 2
    level: block at which we take the output (default to 1, but I tried pretraining on level 0, with downsampled labels...)
    chan1_n: photoreceptor model parameter, default to 9
    
    Returns:
    the model
    
    """
    
    n_filters = [10] + [32*2**i for i in range(depth)]
    
    # inputs
    inputs = layers.Input(shape=(size,size,duration))
    
    #1st version
    #inputs = layers.BatchNormalization()(inputs)
    
    if photo :
        # Saad's photoreceptor layer

        y = Reshape((inputs.shape[-1],inputs.shape[-3]*inputs.shape[-2]))(inputs)

        y = photoreceptor_DA_multichan_randinit(units=chan1_n,kernel_regularizer=l2(1e-4))(y)

        y = Reshape((1,inputs.shape[-3],inputs.shape[-2],chan1_n))(y)
        y = y[:,0,:,:,:]      

        inputs_unet = Activation('relu')(y) 
    
    else:
    
    	inputs_unet = inputs
    
    #2nd version  
    norm_inputs = layers.BatchNormalization()(inputs_unet)
    f_list, p_list = [], [norm_inputs]
    
    for i in range(depth):
        f, p = downsample_block(p_list[-1], n_filters[i])
        f_list.append(f)
        p_list.append(p)
        
    bottleneck = double_conv_block(p_list[-1], n_filters[depth])
    u_list = [bottleneck]
    
    for i in range(level+1):
        u = upsample_block(u_list[-1], f_list[-i-1], n_filters[depth-i-1])
        u_list.append(u)
        
    outputs = layers.Conv2D(digit_max+1, 1, 
                            padding="same", 
                            activation = "softmax")(u_list[-1])
                            
    
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model
    










# Data generator code from https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly (Ines)
#I ended up not using it because I had too many errors with the PR model...












class DataGenerator_frames(tf.keras.utils.Sequence):
    
    def __init__(self, data,labels,
                 n_channels=1,
                 n_classes=11, 
                 batch_size=64, 
                 n_digits = 5, 
                 upsample=True, frame_size = 280, 
                 downsample=False, pool_size=7, strides=7, final_size=4,
                 movie=False, duration=5,
                 digit_max=10,
                 nb_batches = 500,
                 shuffle=True
                ):
        
        
        #'Initialization'
        
        self.n_channels = n_channels
        self.n_classes = n_classes  
        self.batch_size = batch_size
        
        self.n_digits = n_digits
        #n_frames = batch_size
        self.upsample=upsample
        self.frame_size = frame_size
        self.downsample=downsample
        self.pool_size=pool_size
        self.strides=strides
        self.final_size=final_size
        self.movie=movie
        self.duration = duration
        self.digit_max = digit_max
        
        self.nb_batches = nb_batches
        self.shuffle = shuffle
        self.dim = (duration,self.frame_size,self.frame_size)

    
    def __len__(self):
        #'Denotes the number of batches per epoch'
        return int(np.floor(self.nb_batches))
    
    def __getitem__(self, index):


        # Generate data
        X, y = self.__data_generation()

        

        return X, y
        
        
    def __data_generation(self):

            
        X,y = generate_frames_dataset(digits_train, labels_train,
                                      n_digits=self.n_digits, n_frames=self.batch_size, 
                                      upsample=self.upsample, frame_size=self.frame_size,
                                      downsample=self.downsample, pool_size = self.pool_size, strides=self.strides,
                                      final_size = self.final_size,
                                      movie=self.movie, duration=self.duration,
                                      digit_max = self.digit_max                                  
                                      )

        return X, y









class DataGenerator_movies(tf.keras.utils.Sequence):
    
    def __init__(self, data,labels,
                 n_channels=1,
                 n_classes=11, 
                 batch_size=64, duration=5,
                 frame_size=280, n_digits=5,
                 depth=2, level=1,
                 
                shadow = False, shadow_ratio = 0.5, light_intensity = 0.1,
                max_jump=1, max_value=254, speed=1,
                digit_max = 10,

                 nb_batches = 500,
                 shuffle=True
                ):
        
        
        #'Initialization'
        
        self.n_channels = n_channels
        self.n_classes = n_classes  
        self.batch_size = batch_size
        
        #n_movies = batch_size
        self.duration = duration
        self.frame_size = frame_size
        self.n_digits = n_digits
        self.depth = depth
        self.level = level
        self.shadow = shadow
        self.shadow_ratio = shadow_ratio
        self.light_intensity = light_intensity
        self.max_jump = max_jump
        self.max_value = max_value
        self.speed = speed        
        self.digit_max = digit_max
        
        self.nb_batches = nb_batches
        self.shuffle = shuffle
        self.dim = (duration,self.frame_size,self.frame_size)
    
    def __len__(self):
        #'Denotes the number of batches per epoch'
        return int(np.floor(self.nb_batches))
    
    def __getitem__(self, index):


        # Generate data
        X, y = self.__data_generation()
        

        return X, y

        
        
    def __data_generation(self):

            
        X,y = generate_movie_dataset_5(digits_train, labels_train,  
                                             n_movies = self.batch_size, duration = self.duration,
                                            frame_size = self.frame_size, n_digits = self.n_digits,
                                            depth = self.depth, level = self.level,
                                            shadow = self.shadow, shadow_ratio = self.shadow_ratio, 
                                             light_intensity = self.light_intensity,
                                            max_jump=self.max_jump, max_value=self.max_value, speed=self.speed,
                                            digit_max = self.digit_max
                                                      )

        return X, y











#Useful function to put arguments in the python command then get them in the script (Richard)
def read_args(data):
    for arg in sys.argv[1:]:
        if arg.startswith("--") and "=" in arg:
            try:
                name, value, type_var = arg[2:].split("=")
            except:
                raise ValueError(f"Unknown variable format {arg}")	

            if name in data:

                if type_var=="bool":

                    value= value=="True"
                elif type_var=="int":

                    value=int(value)
                elif type_var=="float":

                    value=float(value)

                data[name] = value
            else:
                raise ValueError(f"Unknown variable {name}")
        else:
            raise ValueError(f"Unknown variable format {arg}")

## **Mounting the Drive**


In [ ]:
# Mounting the drive
from google.colab import drive
drive.mount('/content/drive')
path_to_module= '/content/drive/MyDrive/PhD-Nilou/Project#3_DynamicCoding/Nilou_Codes'
import sys
sys.path.append(path_to_module)

Mounted at /content/drive


## **Import the libraries**

In [ ]:
# install libraries 
!pip install matplotlib JSAnimation
!apt install ffmpeg
!pip install -q imageio
!pip install -q imageio-ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for JSAnimation: filename=JSAnimation-0.1-py3-none-any.whl size=11407 sha256=72a381835f7f9ff988cf7262dde3b7e2a85668516860bdd93a793ac76a2dc61a
  Stored in directory: /root/.cache/pip/wheels/1e/16/64/028f540fe8f5eae5026a423bfd88356248074379b79f27b646
Successfully built JSAnimation
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:4.2.7-0ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
# import libraries
# import the libraries 

import tensorflow as tf

import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from JSAnimation import IPython_display
from IPython.display import HTML

import seaborn as sns

import cv2


# from functions2 import *


# Importing necessary libraries
from tensorflow import keras
from tensorflow.keras import Model

from tensorflow.keras.applications.vgg16 import VGG16

# Keras Sequential Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
# Importing all the different layers and optimizers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation, LeakyReLU, TimeDistributed, GlobalAveragePooling1D,Concatenate,Lambda,Input

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
import warnings
warnings.filterwarnings("ignore")

from skimage.draw import line

from scipy.ndimage import gaussian_filter

from matplotlib.animation import FFMpegWriter
from IPython.display import FileLink

## **Load the MNIST Dataset**

In [ ]:
# Load the dataset
## Load MNIST Data

# Set a fixed seed value ( to have the same testing dataset )
np.random.seed(42)
tf.random.set_seed(42) 


num_classes = 10


# image as an input to the CNN model
# input_shape = (28, 28, 1)

# movie as an input with PR model or time-distributed
input_shape = (20, 28, 28)

# movie as an input without PR model 
input_shape_2=(28,28,20)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train_n1 = x_train.astype("float32") / 255
x_test_n1 = x_test.astype("float32") / 255

# make sure images shape is train=(60000, 28, 28, 1) and test=(10000,28,28,1)
x_train_n=np.expand_dims(x_train_n1, -1)
x_test_n=np.expand_dims(x_test_n1, -1)


# #convert class vectors to binary class matrices ( one hot encoded)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11490434/11490434 [==============================] - 0s 0us/step


## **Apply Gaussian Noise to the Data**





In [ ]:
## Use DataGenerator to apply Gaussian Noise

# def add_gaussian_noise(image):
#     noise = np.random.normal(0,0.1, image.shape)
#     noisy_image = image + noise
#     noisy_image = np.clip(noisy_image, 0, 1)
#     return noisy_image

# # Define the ImageDataGenerator with the custom preprocessing function
# datagen = ImageDataGenerator(preprocessing_function=add_gaussian_noise)

# # Initialize an empty array to hold the augmented data
# x_train_augmented = np.empty_like(x_train_n)
# y_train_augmented = np.empty_like(y_train)
# x_test_augmented = np.empty_like(x_test_n)
# y_test_augmented = np.empty_like(y_test)

# # Set the batch size
# batch_size = 128

# # Create a generator that will generate augmented samples from the original dataset
# train_iterator = datagen.flow(x_train_n, y_train, batch_size=batch_size)
# test_iterator = datagen.flow(x_test_n, y_test, batch_size=batch_size)

# # Iterate through the generator and store the augmented samples
# for i, (x_batch, y_batch) in enumerate(train_iterator):
#     start_idx = i * batch_size
#     end_idx = start_idx + x_batch.shape[0]
    
#     x_train_augmented[start_idx:end_idx] = x_batch
#     y_train_augmented[start_idx:end_idx] = y_batch
    
#     if end_idx >= x_train.shape[0]:
#         break

# # Iterate through the generator and store the augmented samples
# for i, (x_batch, y_batch) in enumerate(test_iterator):
#     start_idx = i * batch_size
#     end_idx = start_idx + x_batch.shape[0]
    
#     x_test_augmented[start_idx:end_idx] = x_batch
#     y_test_augmented[start_idx:end_idx] = y_batch
    
#     if end_idx >= x_test.shape[0]:
#         break

In [ ]:
# Define augmented data 

# x_test_aug=x_test_augmented
# y_test_aug=y_test_augmented 
# x_train_aug=x_train_augmented
# y_train_aug=y_train_augmented
# # reshape 
# x_test_aug=np.squeeze(x_test_aug, axis=-1)
# x_train_aug=np.squeeze(x_train_aug, axis=-1)

# **Create a movie with no mask ( Train Dataset)**







In [ ]:
# Create a movie with no mask (with labels)
def create_movie(data,labels,repeats):
  n_samples, img_height, img_width=data.shape
  movie_labels=np.zeros((n_samples, num_classes), dtype=int)
  movie_data=np.zeros((n_samples,repeats,img_height,img_width,1),dtype=np.float32)
  for i in range(n_samples):
    movie_data[i]=np.repeat(data[i][np.newaxis, :, :, np.newaxis], repeats, axis=0)

  movie_labels[i] = labels[i]
  return movie_data,movie_labels

In [ ]:
# Augmented Data/noisy image: create movie for train dataset 
# # number of frames 
# repeats=20
# x_train_movie, movie_labels_train=create_movie(x_train_aug,y_train_aug,repeats)
# x_train_movie_sq=np.squeeze(x_train_movie, axis=-1)
# # reshape
# x_train_movie_reshaped=np.transpose(x_train_movie_sq,(0,2,3,1))

In [ ]:
# Clean images: create movie for train dataset 
# number of frames 
repeats=20
x_train_movie, movie_labels_train=create_movie(x_train,y_train,repeats)
x_train_movie_sq=np.squeeze(x_train_movie, axis=-1)
# reshape
x_train_movie_reshaped=np.transpose(x_train_movie_sq,(0,2,3,1))

In [ ]:
# label per frame 
y_train_frames=np.repeat(y_train,repeats, axis=0)

## **Visualize the train dataset**

In [ ]:

# Function to update the frames in the animation ( reshaped dataset )
# def update(i):
#     im.set_array(x_train_movie_sq[movie_index, i, :, :])
#     # frame_label = str(np.argmax(y_train[movie_index * repeats + i]))
    
#     frame_label = str(np.argmax(y_train_frames[movie_index* repeats + i]))
#     ax.set_title(f"Frame {i+1}, True label: {frame_label}")
#     return [im]

# # Choose which movie in the dataset to visualize
# movie_index = 7

# # true_label=str(np.argmax(y_train[movie_index]))
# true_label = str(np.argmax(y_train_frames[movie_index * repeats + i]))

# # Create the initial plot
# fig, ax = plt.subplots()
# im = ax.imshow(x_train_movie_sq[ movie_index, 0, :, :], interpolation='none')

# # plt.axis('off')

# # set the title with the true label 
# ax.set_title(f"True label: {true_label}")

# # Create the animation
# ani = FuncAnimation(fig, update, frames=20, interval=200, blit=True)

# # Display the animation
# HTML(ani.to_jshtml())

In [ ]:
# save animation as an MP4 video file
writer=FFMpegWriter(fps=15,metadata=dict(artist='Me'),bitrate=1800)
ani.save("animation12.mp4",writer=writer)
animation_path = '/content/drive/MyDrive/PhD-Nilou/Project#3_DynamicCoding/Nilou_Codes/Animations/Train Dataset/animation12.mp4'
ani.save(animation_path, writer=writer)

NameError: ignored

In [ ]:
# the index of the sample you want to display
sample_index =300
# the number of frames in each image
num_frames =20
repeats=20
fig, axs = plt.subplots(1, num_frames, figsize=(50, 30), subplot_kw={'aspect': 'equal'})

# loop through the frames and display each one separately
for i in range(num_frames):
    frame_label = str(np.argmax(y_train_frames[sample_index * repeats + i]))
    axs[i].set_title(f"Frame {i+1}, True label: {frame_label}")
    axs[i].imshow(x_train_movie[sample_index, i, :, :, 0])
    
    

    # axs[i].axis('off')

plt.show()

# **Applying Masks to the Test Dataset**





## **Binary mask**


In [ ]:
## Binary Mask 
def modify_brightness_mask7(img,spotlight_intensity):
  image_shape = img.shape

  # Create a binary mask with the same shape as an MNIST image
  mask_2d = np.random.randint(0,5, size=image_shape)

  # Shuffle the indices in the 2D mask
  shuffled_indices = np.random.permutation(np.arange(mask_2d.size))

  # Apply the shuffled indices to the 2D mask ( flatten into 1D)
  shuffled_mask = mask_2d.reshape(-1)[shuffled_indices]

  # Reshape the shuffled mask back to the original image shape
  shuffled_mask_2d = shuffled_mask.reshape(image_shape)



      # Apply Gaussian blur to the mask (reduce sharp edges)
  blurred_mask = gaussian_filter(shuffled_mask_2d, sigma=1)

    # Normalize the blurred mask to the range [0, 1]
  normalized_blurred_mask = blurred_mask / np.max(blurred_mask)
  
  # Normalize input data 
  normalized_img = img / 255.0

    # Apply the mask to the image using element-wise addition
  masked_image = normalized_img + normalized_blurred_mask

    # Clip the values of the masked_image to the range [0, 1]
  masked_image = np.clip(masked_image, 0, 1)

    # Rescale the masked_image back to the range [0, 255]
  masked_image = (masked_image * 255).astype(np.uint8)

  # apply mask
  img_mod = img.copy()
  img_mod=img_mod*((masked_image*spotlight_intensity)+1)
      
  return img_mod

## **Binary mask2**




In [ ]:
# Rectangular mask 

# def modify_brightness_mask8(img,spotlight_intensity):
#     image_shape = img.shape


#      # Generate random threshold value between 0.2 and 0.8
#     threshold = np.random.uniform(0.1,0.8)
    
#     # Generate random mask with threshold value
#     mask_rand = np.random.rand(*image_shape) < threshold
    
#     # # Apply mask to image
#     # masked_image = np.where(mask_rand, x_test[i], 0)

#     img_mod= ((mask_rand*10)+1)*img

#     return img_mod
    


## **Circular mask**

In [ ]:
# # circular mask 

# def modify_brightness_mask6(img, spotlight_intensity):

    
#     # Generate random circle 
#     cx = np.random.randint(4, 24)
#     cy = np.random.randint(4, 24)
#     r = np.random.randint(4,12)

#     # create a mask 
#     mask_circ = np.zeros_like(img)
#     y, x = np.ogrid[-cx:img.shape[0]-cx, -cy:img.shape[1]-cy] # adjust size of mask to match image
#     mask_circ[(x**2 + y**2) <= r**2] = 1

#     # Apply the mask to the image
#     img_mod2 = img.copy()
#     masked_img_circ = ((mask_circ * spotlight_intensity) + 1) * img_mod2

#     return masked_img_circ


# **Create a movie with masked images for the testing the model**

## **Condition1: A movie with the mask being generated for the last 5 frames**





In [ ]:
# # Condition 1: Create a movie with the mask being generated for the last 5 frames 
# def create_movie_with_mask_last(data, labels, repeats, mask_frames, spotlight_intensity):
#     n_samples, img_height, img_width = data.shape
#     movie_data = np.zeros((n_samples, repeats, img_height, img_width, 1), dtype=np.float32)
#     movie_labels = np.zeros((n_samples, num_classes), dtype=int)
#     mask_frames_start = repeats - mask_frames
#     for i in range(n_samples):
#         # inverted_data=255-data[i]
#         # increase the brightness of the digit
#         # BK_int=10
#         # D_int=200
#         # brighter_data=np.clip(data[i]+D_int,0,255)
#         # brighter_data[data[i]==0]=BK_int
#         mask = None
#         for j in range(repeats):
#             if mask_frames_start <= j < repeats:
#                 if mask is None:
#                     mask = modify_brightness_mask7(data[i], spotlight_intensity)
#                 movie_data[i, j, :, :, 0] = mask
#             else:
#                 movie_data[i, j, :, :, 0] = data[i]

#         movie_labels[i] = labels[i]

#     return movie_data, movie_labels

## **Condition2: A movie with the mask being generated for 5 consecutive frames/the starting frame of the mask is different for each movie**

In [ ]:
# Condition 2: Create a movie with the mask being generated seperately for each of the 5 consecutive frames ( to keep the mask and brightness the same for 5 frames)
import time

def create_movie_with_mask(data, labels, repeats, mask_frames, spotlight_intensity):
    n_samples, img_height, img_width = data.shape
    movie_data = np.zeros((n_samples, repeats, img_height, img_width, 1), dtype=np.float32)
    movie_labels = np.zeros((n_samples, num_classes), dtype=int)
    category_labels = np.zeros(n_samples, dtype=int)
    num_categories = repeats - mask_frames + 1
    
    for i in range(n_samples):
        # Choose a random starting frame for this sample
        start_frame = np.random.randint(0, repeats - mask_frames + 1)
        category_labels[i] = start_frame
        
        # Create a mask for this sample
        mask_start = np.random.randint(0, repeats - mask_frames + 1)
        mask = None
        for j in range(repeats):
            if j >= mask_start and j < mask_start + mask_frames:
                if mask is None:
                    mask = modify_brightness_mask7(data[i], spotlight_intensity)
                movie_data[i, j, :, :, 0] = mask
            else:
                movie_data[i, j, :, :, 0] = data[i]
        
        movie_labels[i] = labels[i]
    
    return movie_data, movie_labels, category_labels


## **Train Dataset with mask**



In [ ]:
# Masked images for train dataset: to have masked images for the last 5 five frames (Train Dataset)
# # for shadow divide it by something
# spotlight=10000
# mask_frames=3
# repeats=20
# x_train_mask_movie, movie_labels_mask_train, category_labels=create_movie_with_mask(x_train,y_train, repeats,mask_frames,spotlight)
# # reshape
# x_train_mask_movie_sq=np.squeeze(x_train_mask_movie, axis=-1)
# x_train_mask_movie_reshaped=np.transpose(x_train_mask_movie_sq, (0,2,3,1))

## **Test Dataset with mask**






### **Augmented Data/Gaussian Noise**

In [ ]:
# DO NOT RUN THIS: ## to have masked images for the last 5 five frames (Test Dataset)-Augmented data 
# for shadow divide it by something
# spotlight=2
# mask_frames=3
# repeats=20
# x_test_mask_movie, movie_labels_mask_test, category_labels=create_movie_with_mask(x_test_aug,y_test_aug, repeats,mask_frames,spotlight)
# # reshape
# x_test_mask_movie_sq=np.squeeze(x_test_mask_movie, axis=-1)
# x_test_mask_movie_reshaped=np.transpose(x_test_mask_movie_sq, (0,2,3,1))


### **Data without Gaussian Noise**

In [ ]:
## to have masked images for the last 5 five frames (Test Dataset)
# for shadow divide it by something
spotlight=2
mask_frames=6
repeats=20
x_test_mask_movie, movie_labels_mask_test, category_labels=create_movie_with_mask(x_test,y_test, repeats,mask_frames,spotlight)
# reshape
x_test_mask_movie_sq=np.squeeze(x_test_mask_movie, axis=-1)
x_test_mask_movie_reshaped=np.transpose(x_test_mask_movie_sq, (0,2,3,1))

In [ ]:
## test dataset (labels)
movie_labels_mask_test_frames=np.repeat(movie_labels_mask_test,repeats,axis=0)
## train dataset (labels)
# movie_labels_mask_train_frames=np.repeat(movie_labels_mask_train,repeats,axis=0)

In [ ]:
# assuming x_test_mask_movie[1] is your input array ( test dataset ) for CNN model 
x_test_mask_movie_reshaped = np.transpose(x_test_mask_movie, (0,2,3,1,4))
x_test_mask_movie_reshaped = np.squeeze(x_test_mask_movie_reshaped)  # remove the last singleton dimension

In [ ]:
y_test_frames = np.repeat(y_test,repeats, axis=0)
y_train_frames=np.repeat(y_train,repeats, axis=0)

# **Visualize Train Dataset-with mask**

In [ ]:
# Function to update the frames in the animation ( initial test data )
# def update(i):
#     im.set_array(x_train_mask_movie_sq[movie_index, i, :, :])
#     frame_label = str(np.argmax(y_train_frames[movie_index * repeats + i]))
#     ax.set_title(f"Frame {i+1}, True label: {frame_label}")
#     return [im]

# # Choose which movie in the dataset to visualize
# movie_index =5

# # True label 
# true_label=str(np.argmax(movie_labels_mask_train_frames[movie_index]))

# # Create the initial plot
# fig, ax = plt.subplots()
# im = ax.imshow(x_train_mask_movie[movie_index, 0, :, :, 0],interpolation='none')
# # plt.axis('off')

# # set the title with the true label 
# ax.set_title(f"True label: {true_label}")

# # Create the animation
# ani = FuncAnimation(fig, update, frames=20, interval=200 , blit=True)

# # Display the animation
# HTML(ani.to_jshtml())

# **Visualize Test Dataset with mask**


In [ ]:
# Function to update the frames in the animation ( initial test data )
def update(i):
    im.set_array(x_test_mask_movie_sq[movie_index, i, :, :])
    frame_label = str(np.argmax(y_test_frames[movie_index * repeats + i]))
    ax.set_title(f"Frame {i+1}, True label: {frame_label}")
    return [im]

# Choose which movie in the dataset to visualize
movie_index =1

# True label 
true_label=str(np.argmax(movie_labels_mask_test_frames[movie_index]))

# Create the initial plot
fig, ax = plt.subplots()
im = ax.imshow(x_test_mask_movie[movie_index, 0, :, :, 0],interpolation='none')
# plt.axis('off')

# set the title with the true label 
ax.set_title(f"True label: {true_label}")

# Create the animation
ani = FuncAnimation(fig, update, frames=20, interval=200 , blit=True)

# Display the animation
HTML(ani.to_jshtml())

In [ ]:
# save animation as an MP4 video file
writer=FFMpegWriter(fps=15,metadata=dict(artist='Me'),bitrate=1800)
ani.save("animation_mask49.mp4",writer=writer)
animation_path = '/content/drive/MyDrive/PhD-Nilou/Project#3_DynamicCoding/Nilou_Codes/Animations/Test Dataset/animation_mask49.mp4'
ani.save(animation_path, writer=writer)

In [ ]:
# Create a list of images to display in the animation
num_frames=20
movie_index=1
frames = [x_test_mask_movie[movie_index, i, :, :, 0] for i in range(num_frames)]

# Create the animation
ani = animation.FuncAnimation(fig, func=lambda i: [ax.imshow(frames[i])], frames=num_frames, interval=200, blit=True)

# Display the animation
HTML(ani.to_jshtml())

In [ ]:
# save animation as an MP4 video file
writer=FFMpegWriter(fps=15,metadata=dict(artist='Me'),bitrate=1800)
ani.save("animation_mask48.mp4",writer=writer)
animation_path = '/content/drive/MyDrive/PhD-Nilou/Project#3_DynamicCoding/Nilou_Codes/Animations/Test Dataset/animation_mask48.mp4'
ani.save(animation_path, writer=writer)

In [ ]:
# the index of the sample you want to display
sample_index =1
# the number of frames in each image
num_frames =20
repeats=20
fig, axs = plt.subplots(1, num_frames, figsize=(50, 30), subplot_kw={'aspect': 'equal'})

# loop through the frames and display each one separately
for i in range(num_frames):
    frame_label = str(np.argmax(y_test_frames[sample_index * repeats + i]))
    axs[i].set_title(f"Frame {i+1}, True label: {frame_label}")
    axs[i].imshow(x_test_mask_movie[sample_index, i, :, :, 0])
    
    

    # axs[i].axis('off')

plt.show()

# **Intensity of each pixel & Convolution Operation**

In [ ]:
# Intensity of each pixel

# sample_index=100
# rows, cols=3,3

# for i in range(rows):
#     for j in range(cols):
#         # Extract pixel intensities for the given sample and pixel coordinates
#         pixel_intensities = x_test_mask_movie[sample_index, :, i, j, 0]

#         # Create a new figure
#         plt.figure()

#         # Plot pixel intensities
#         plt.plot(pixel_intensities)
#         plt.xlabel('Frame')
#         plt.ylabel('Intensity')
#         plt.title(f'Intensity of pixel ({i}, {j}) over 20 frames')
#         plt.grid(True)

In [ ]:
## Intensity of all pixels 
# sample_index=100
# rows, cols=28,28

# # Create a new figure with a specified size
# fig, axes = plt.subplots(rows, cols, figsize=(cols, rows), dpi=100)

# # Set overall title for the figure
# fig.suptitle(f'Pixel intensities over 20 frames for sample {sample_index}')

# for i in range(rows):
#   for j in range(cols):
      
#       pixel_intensities= (x_test_mask_movie[100,: ,i, j, 0])
#       conv_output = np.convolve(pixel_intensities, ky, mode='full')

#         # Plot pixel intensities on the corresponding subplot
#       ax = axes[i, j]
#       ax.plot(conv_output)
#       ax.set_xticks([])  # Hide x-axis ticks
#       ax.set_yticks([])  # Hide y-axis ticks

# # Adjust the space between subplots
# plt.subplots_adjust(wspace=0.1, hspace=0.1)
      


In [ ]:
# inputs = Input(shape=input_shape)
# tau_y=0.02
# n_y=0.03
# gamma_value=0.07
# tau_z=0.07
# n_z=0.09
# t = tf.range(0,inputs.shape[1],dtype='float32')
# multFunc_tau=100;
# multFunc_n=10;
# multFunc_g=10;

# ky = generate_simple_filter_multichan(np.array([tau_y],dtype=np.float32)*multFunc_tau, np.array([n_y],dtype=np.float32)*multFunc_n, t)
# kz=generate_simple_filter_multichan(np.array([tau_z],dtype=np.float32)*multFunc_tau, np.array([n_z],dtype=np.float32)*multFunc_n, t)
# kz=(gamma_value*multFunc_g*ky)+(1-gamma_value*multFunc_g)*kz

In [ ]:
# plt.plot( ky)

In [ ]:
# pixel_intensities = x_test_mask_movie[100, :, 0, 2, 0]
# pixel_intensities.shape

In [ ]:
# if isinstance(ky, tf.Tensor):
#     ky = ky.numpy()

# if isinstance(kz, tf.Tensor):
#     kz = kz.numpy()

# ky=np.squeeze(ky)

In [ ]:

# Squeeze ky to ensure it's a 1-dimensional array
# ky = np.squeeze(ky)



# pixel_intensities = x_test_mask_movie[100, :, 0, 2, 0]

# conv_output = np.convolve(pixel_intensities, kz, mode='full')
# plt.figure()
# # plt.plot(ky*100, label='fast filter')
# plt.plot(kz*100, label='slow filter')
# plt.plot(pixel_intensities, label='Pixel intensities')
# plt.plot(conv_output, label='Convolution output')
# plt.xlabel('Frame')
# plt.ylabel('Value')
# plt.title('Convolution between pixel intensities and filter ky')
# plt.legend()
# plt.grid(True)
# plt.show()

In [ ]:
# save animation as an MP4 video file
writer=FFMpegWriter(fps=15,metadata=dict(artist='Me'),bitrate=1800)
ani.save("animation_mask46.mp4",writer=writer)
animation_path = '/content/drive/MyDrive/PhD-Nilou/Project#3_DynamicCoding/Nilou_Codes/Animations/Test Dataset/animation_mask46.mp4'
ani.save(animation_path, writer=writer)

# **Size of test and train datasets**


In [ ]:
# # Plot a bar chart of the sizes

train_size=x_train_movie.shape[0]
test_size=x_test_mask_movie.shape[0]
# train_repeated_size=x_train_repeated.shape[0]
plt.bar(['Train', 'Test'], [train_size, test_size])
plt.xlabel('Dataset')
plt.ylabel('Size')
plt.show()

# **Models**

## **CNN+PR Model for an image**





In [ ]:
#CNN + PR Model ( Time independent )


# from tensorflow.keras.regularizers import l1_l2
# ## PR+ CNN Model we have 20 samples of 20 
# input_shape = (20, 28, 28)

# # define inputs tensor
# inputs = Input(shape=input_shape)

# chan1_n=1

# # PR Architecture ( without time_distributed)- the code is modified so this would get an error
# y1 = Reshape((inputs.shape [1],inputs.shape [-2]*inputs.shape [-1]))(inputs)
# y1 = photoreceptor_DA_multichan_randinit(units=chan1_n,kernel_regularizer=l2(1e-4))(y1)
# y1 = Reshape((1,inputs.shape[-2],inputs.shape[-1],chan1_n))(y1)
# y1 = y1[:,0,:,:,:]  

# y2=Activation('relu')(y1)
# y2=Conv2D(32,kernel_size=(3,3), kernel_regularizer=l1_l2(l1=0.001, l2=0.001))(y2)
# y2=(LayerNormalization())(y2)
# y2=Activation('relu')(y2)
# y2=MaxPooling2D(pool_size=(2,2))(y2)
# y2=Conv2D(64,kernel_size=(3,3))(y2)
# y2=LayerNormalization()(y2)
# y2=Activation('relu')(y2)
# y2=MaxPooling2D(pool_size=(2,2))(y2)
# y2=Flatten()(y2)
# y2=Dropout(0.5)(y2)
# y2=Dense(num_classes,activation="softmax")(y2)

# # # Define PR+CNN model
# model=tf.keras.Model(inputs=inputs,outputs=y2)

# # Define the submodule consisting of layers 1 to 4
# submodel_PR = tf.keras.Model(inputs=inputs, outputs=y1)


In [ ]:
# model.summary()

In [ ]:
# from keras.utils.vis_utils import plot_model
# model=model
# plot_model(model,to_file='Model.png')

### **Comiling and training CNN+PR Model (time-independent)**

In [ ]:
# # PR+CNN Time independent 
# n=100

# model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])
# history=model.fit(x_train_movie_sq,y_train_aug,batch_size=128,epochs=n,validation_data=(x_test_mask_movie_sq,y_test_aug))

In [ ]:
# Results
### PR + CNN Model (without time_distributed)

# dict_hist = history.history # assuming you have already trained your model and stored the history object

# list_ep = [i for i in range(1,n+1)]

# plt.figure(figsize=(8,8))

# plt.subplot(2, 1, 1) # adding a subplot for accuracy
# plt.plot(list_ep, dict_hist['accuracy'],ls='--',label='accuracy-train_dataset')
# plt.plot(list_ep, dict_hist['val_accuracy'],ls='--',label='accuracy-validation_dataset')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.subplot(2, 1, 2) # adding a subplot for loss
# plt.plot(list_ep, dict_hist['loss'],ls='--',label='loss-train_dataset')
# plt.plot(list_ep, dict_hist['val_loss'],ls='--',label='loss-validation_dataset')
# plt.ylabel('Loss')
# plt.xlabel('Epochs')
# plt.legend()

# plt.show()

### **Accuracy Plot for PR+CNN Model- Time independent**

In [ ]:
# y_pred_new = submodel_PR.predict(x_test_mask_movie_sq)

# sample_index = 1000  # Choose the index of the sample you want to visualize

# plt.figure(figsize=(5, 5))
# plt.imshow(y_pred_new[sample_index, :, :])
# plt.title(f"Output of the PR model - Sample {sample_index}")
# plt.colorbar()
# plt.show()

## **PR+CNN Model- Time Distributed**

In [ ]:
#CNN + PR Model (Time Distributed)


from tensorflow.keras.regularizers import l1_l2
## PR+ CNN Model 
input_shape = (20, 28, 28)

# define inputs tensor
inputs = Input(shape=input_shape)

chan1_n=20

# PR Architecture 
y1 = Reshape((inputs.shape [1],inputs.shape [-2]*inputs.shape [-1]))(inputs)
y1 = photoreceptor_DA_multichan_randinit(units=chan1_n,kernel_regularizer=l2(1e-4))(y1)
y1 = Reshape((inputs.shape[1],inputs.shape[-2],inputs.shape[-1],chan1_n))(y1)
# leave the first 5 frames 
y1 = y1[:,:,:,:,:]



y2=TimeDistributed(keras.layers.Conv2D(32,kernel_size=(3,3), activation='relu'))(y1)
y2=TimeDistributed(keras.layers.LayerNormalization())(y2)
y2=TimeDistributed(keras.layers.MaxPooling2D(pool_size=(2,2)))(y2)
y2=TimeDistributed(keras.layers.Conv2D(64,kernel_size=(3,3), activation='relu'))(y2)
y2=TimeDistributed(keras.layers.LayerNormalization())(y2)
y2=TimeDistributed(keras.layers.MaxPooling2D(pool_size=(2,2)))(y2)
y2=TimeDistributed(keras.layers.Flatten())(y2)
# y2=keras.layers.GlobalAveragePooling1D()(y2)
y2=keras.layers.Dropout(0.5)(y2)
y2=keras.layers.Dense(num_classes,activation="softmax")(y2)


# # Define PR+CNN model
model2=tf.keras.Model(inputs=inputs,outputs=y2)

# Define the submodule consisting of layers 1 to 4
submodel_PR2 = tf.keras.Model(inputs=inputs, outputs=y1)

In [ ]:
from keras.utils.vis_utils import plot_model
model=model2
plot_model(model2,to_file='Model1.png')

### **Data Generator: label/frame**

In [ ]:
# 1. Verify data and labels alignment:

# Assuming x_data and y_labels are your loaded data and labels
num_samples = 5  # Number of random samples to visualize
frame_index=0

for _ in range(num_samples):
    idx = np.random.randint(0, len(x_train_movie_sq))
    movie = x_train_movie_sq[idx]
    frame=movie[frame_index]
    label = np.argmax(y_train[idx])

        # Visualize the sample and its label (assuming 28x28 images)
    plt.imshow(frame.reshape(28, 28))
    plt.title(f"Label: {label}")
    plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_movie_frames(x, y, movie_index):
    movie_data = x[movie_index]
    movie_labels = np.argmax(y[movie_index])

    frames = 20  # Change this to match the number of frames per movie

    # Get the digit label for the whole movie sequence
    digit_label = (movie_labels)

    # Iterate through the frames and visualize them with their corresponding label
    for i in range(frames):
        frame = movie_data[i]

        # Reshape digit to (28, 28) for visualization
        frame = frame.reshape(28, 28)

        plt.imshow(frame)
        plt.title(f"Frame {i+1}, Digit Label: {digit_label}")
        plt.show()

# Call the function with the desired movie_index
movie_index = 100  # Change this to the index of the movie you want to visualize
visualize_movie_frames(x_test_mask_movie_sq, y_test, movie_index)

In [ ]:
def movie_level_data_generator(x, y, batch_size):
    n_samples = len(x)
    indices = np.arange(n_samples)
    frames = 20

    while True:
        np.random.shuffle(indices)

        for start in range(0, n_samples, batch_size):
            end = min(start + batch_size, n_samples)
            batch_idx = indices[start:end]

            batch_x = np.array([x[i] for i in batch_idx])
            batch_y = y[batch_idx]

            # Repeat the labels for each frame in the movie
            batch_y = np.repeat(batch_y, frames, axis=0)
            
            # Reshape the labels to match the model's output shape
            batch_y = batch_y.reshape(-1, frames, 10)

            # Print the frame and its corresponding label for each movie in the batch
            for movie_idx, movie_data in enumerate(batch_x):
                for i in range(frames):
                    frame = movie_data[i, :, :]
                    label = batch_y[movie_idx, i, :]

                    digit = frame
                    digit_label = np.argmax(label)

                    # Reshape digit to (28, 28) for visualization
                    digit = digit.reshape(28, 28)

                    # plt.imshow(digit)
                    # plt.title(f"Movie {movie_idx+1}, Frame {i+1}, Digit Label: {digit_label}")
                    # plt.show()

            yield batch_x, batch_y


In [ ]:
train_generator = movie_level_data_generator(x_train_movie_sq, y_train, batch_size=16)
x, y = next(train_generator)
validation_generator = movie_level_data_generator(x_test_mask_movie_sq, y_test, batch_size=16)
x, y = next(validation_generator)
batch_size = 16
steps_per_epoch = len(x_train_movie_sq) // batch_size
validation_steps = len(x_test_mask_movie_sq) // batch_size

### **Compiling and Training**

In [ ]:
# one label per movie
# one label per digit 

# n=30

# model2.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])
# history2=model2.fit(x_train_movie_sq,y_train_aug,batch_size=128,epochs=n,validation_data=(x_test_mask_movie_sq,y_test_aug))

In [ ]:
# one label per frame
n = 10
model2.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])
history2 = model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=n, 
                      validation_data=validation_generator, validation_steps=validation_steps,
                      verbose=1)

### **Accuracy Plot (label/frame & label/movie)**

In [ ]:
## Compare the accuracy of the CNN+PR model 

import matplotlib.pyplot as plt
import numpy as np


# convert to one-hot encoded 
y_test=np.argmax(y_test,axis=1)

frames_to_exclude = 2
frame_num = 20 - frames_to_exclude
frame_accs = np.zeros(frame_num)
num_movies_all_correct = np.zeros(frame_num)

# Loop through each movie in the test set
for movie_idx, movie_data in enumerate(x_test_mask_movie_sq):
    
    movie_labels = y_test[movie_idx]
    movie_labels = np.repeat(movie_labels, frame_num+frames_to_exclude, axis=0)
    movie_labels = movie_labels.reshape(frame_num+frames_to_exclude, -1)
    
    # Predict the labels for each frame in the movie
    predicted_labels = model2.predict(movie_data.reshape(1, frame_num+frames_to_exclude, 28, 28))
    
    # Calculate the accuracy for each frame in the movie, starting from the frame after those to exclude
    for frame_idx in range(frames_to_exclude, frame_num+frames_to_exclude):
        frame_acc = (np.argmax(predicted_labels[0][frame_idx]) == (movie_labels[frame_idx]))
        frame_accs[frame_idx - frames_to_exclude] += int(frame_acc)
        
        # Display the digit image and its predicted label

        # Repeat the label for each frame in the movie
        label = movie_labels[frame_idx]
        # Print the accuracy for this frame
        print(f"Movie {movie_idx+1}, Frame {frame_idx} Accuracy: {frame_acc}, predicted_label: {np.argmax(predicted_labels[0][frame_idx])}, actual label: {(movie_labels[frame_idx])}")


                    # Check if all frames match the labels
    all_correct = all(np.argmax(predicted_labels[0][i]) == (movie_labels[i]) for i in range(frames_to_exclude, 20))
    if all_correct:
        num_movies_all_correct += 1

# Print the number of movies where all frames match the labels
print(f"Number of movies where all frames match the labels: {num_movies_all_correct[0]}")
    
# Divide by the number of movies to get the mean accuracy for each frame
frame_accs /= len(x_test_mask_movie_sq)

In [ ]:

n1=10

dict_hist = history2.history # assuming you have already trained your model and stored the history object

list_ep = [i for i in range(1,n1+1)]

plt.figure(figsize=(8,8))

plt.subplot(2, 1, 1) # adding a subplot for accuracy
plt.plot(list_ep, dict_hist['accuracy'],ls='--',label='accuracy-train_dataset')
plt.plot(list_ep, dict_hist['val_accuracy'],ls='--',label='accuracy-validation_dataset')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 1, 2) # adding a subplot for loss
plt.plot(list_ep, dict_hist['loss'],ls='--',label='loss-train_dataset')
plt.plot(list_ep, dict_hist['val_loss'],ls='--',label='loss-validation_dataset')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()

plt.show()

### **Visualize Predicted Labels**

In [ ]:
# Choose a movie index to plot its frames
movie_idx = 9999

# Get the data and labels for the chosen movie
movie_data = x_test_mask_movie_sq[movie_idx]
movie_labels = y_test[movie_idx]

# Repeat the labels for each frame in the movie
frame_num = 20
movie_labels = np.repeat(movie_labels, frame_num, axis=0)
movie_labels = movie_labels.reshape(frame_num, -1)

# Predict the labels for each frame in the movie
predicted_labels = model2.predict(movie_data.reshape(1, 20, 28, 28))

# Loop through each frame in the movie and plot it with its predicted label
for frame_idx in range(frame_num):
    # Display the digit image and its predicted label
    digit = movie_data[frame_idx, :, :]
    digit_label = np.argmax(predicted_labels[0][frame_idx])
    digit = digit.reshape(28, 28)
    plt.imshow(digit)
    plt.title(f"Movie {movie_idx+1}, Frame {frame_idx+1}, Digit Label: {digit_label}")
    plt.show()

In [ ]:
# Actual digit vs predicted image 
movie_idx = 9199
# the number of frames in each image
num_frames =18

# Get the data and labels for the chosen movie
movie_data = x_test_mask_movie_sq[movie_idx]
movie_labels = y_test[movie_idx]

# Repeat the labels for each frame in the movie
frame_num = 18
movie_labels = np.repeat(movie_labels, frame_num, axis=0)
movie_labels = movie_labels.reshape(frame_num, -1)

# Predict the labels for each frame in the movie
predicted_labels = model2.predict(movie_data.reshape(1, 20, 28, 28))

fig, axs = plt.subplots(1, frame_num , figsize=(100, 30), subplot_kw={'aspect': 'equal'})
# loop through the frames and display each one separately
for i in range(frame_num):
    digit = movie_data[i+2, :, :]
    digit_label = np.argmax(predicted_labels[0][i+2])
    digit = digit.reshape(28, 28)
    axs[i].imshow(digit)
    axs[i].set_title(f"Movie {movie_idx+1}, Frame {i+3}, Digit Label: {digit_label}")
    axs[i].axis('off')
  



plt.show()
   



### **Input to the mode**

In [ ]:
### Input 
# the index of the sample you want to display
sample_index = 9199
# the number of frames in each image
num_frames = 20

fig, axs = plt.subplots(1, num_frames, figsize=(50, 30), subplot_kw={'aspect': 'equal'})

# loop through the frames and display each one separately
for i in range(num_frames):
    axs[i].imshow(x_test_mask_movie[sample_index, i, :, :])
    axs[i].set_title(f"Movie {sample_index + 1}, Frame {i + 1}")
    axs[i].axis('off')

plt.show()

### **Output of PR Model**

In [ ]:
# ## output of the PR Model 
# # Check the output of PR ( time series )
# output=submodel_PR2.predict(x_test_mask_movie_sq)

# # the index of the sample you want to display
# sample_index = 6789
# # the number of frames in each image
# num_frames = 20

# fig, axs = plt.subplots(1, num_frames, figsize=(50, 30), subplot_kw={'aspect': 'equal'})

# # loop through the frames and display each one separately
# for i in range(num_frames):
#     axs[i].imshow(output[sample_index, i, :, :,0])
#     axs[i].set_title(f"Movie {sample_index + 1}, Frame {i + 1}")
#     axs[i].axis('off')

# plt.show()

## **CNN Model- Time independent**

In [ ]:
# Model 1: Define CNN Model for a movie ( CNN_input shape= 28,28,20) (time independent )
model1=keras.Sequential(
    [
        keras.Input(shape=input_shape_2),
        keras.layers.Conv2D(32,kernel_size=(3,3)),
        keras.layers.LayerNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Conv2D(64,kernel_size=(3,3)),
        keras.layers.LayerNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes,activation="softmax")
    ]
)



In [ ]:
from keras.utils.vis_utils import plot_model
model=model1
plot_model(model1,to_file='Model1.png')

### **Compiling and Training**

In [ ]:
# # # compile and train the model # 1 (CNN model for a movie)
# n1=100
# model1.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001),metrics=['accuracy'])
# history1=model1.fit(x_train_movie_reshaped,y_train_aug,batch_size=128,epochs=n1,validation_data=(x_test_mask_movie_reshaped,y_test_aug))

### **Accuracy Plot**

In [ ]:

# dict_hist = history1.history # assuming you have already trained your model and stored the history object

# list_ep = [i for i in range(1,n1+1)]

# plt.figure(figsize=(8,8))

# plt.subplot(2, 1, 1) # adding a subplot for accuracy
# plt.plot(list_ep, dict_hist['accuracy'],ls='--',label='accuracy-train_dataset')
# plt.plot(list_ep, dict_hist['val_accuracy'],ls='--',label='accuracy-validation_dataset')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.subplot(2, 1, 2) # adding a subplot for loss
# plt.plot(list_ep, dict_hist['loss'],ls='--',label='loss-train_dataset')
# plt.plot(list_ep, dict_hist['val_loss'],ls='--',label='loss-validation_dataset')
# plt.ylabel('Loss')
# plt.xlabel('Epochs')
# plt.legend()

# plt.show()

## **CNN Model- Time Distributed**

In [ ]:
## Modified CNN model for a sequence of images/movies: Model 3 ( CNN Time Distributed)
input_shape = (28, 28, 1)
num_frames=20

model3=keras.Sequential(
    [
        keras.Input(shape=(num_frames, input_shape[0], input_shape[1],1)),
        TimeDistributed(keras.layers.Conv2D(32,kernel_size=(3,3), activation='relu')),
        TimeDistributed(keras.layers.LayerNormalization()),
        TimeDistributed(keras.layers.MaxPooling2D(pool_size=(2,2))),
        TimeDistributed(keras.layers.Conv2D(64,kernel_size=(3,3), activation='relu')),
        TimeDistributed(keras.layers.LayerNormalization()),
        TimeDistributed(keras.layers.MaxPooling2D(pool_size=(2,2))),
        TimeDistributed(keras.layers.Flatten()),
        # keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes,activation="softmax")

    ]
)

In [ ]:
from keras.utils.vis_utils import plot_model
model=model3
plot_model(model3,to_file='Model3.png')

### **Compiling and Training**

In [ ]:
# Compile and train model 3 ( label/movie)
# n1=30
# model3.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001),metrics=['accuracy'])
# history3=model3.fit(x_train_movie,y_train_aug,batch_size=128,epochs=n1,validation_data=(x_test_mask_movie,y_test_aug))

In [ ]:
# Label/frame
n = 10
model3.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.00001), metrics=['accuracy'])
history3 = model3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=n, 
                      validation_data=validation_generator, validation_steps=validation_steps,
                      verbose=1)

### **Accuracy Plot** *italicized text*

In [ ]:
# import matplotlib.pyplot as plt
frame_num=20
num_movies_all_correct=np.zeros(20)
frame_accs=np.zeros(20)
mismatched_movies=[]
# Loop through each movie in the test set
for movie_idx, movie_data in enumerate(x_test_mask_movie_sq):
    
    movie_labels = (y_test[movie_idx])
    movie_labels = np.repeat(movie_labels, frame_num, axis=0)
    movie_labels = movie_labels.reshape(frame_num, -1)
    
    # Predict the labels for each frame in the movieslack
    
    predicted_labels = model3.predict(movie_data.reshape(1, 20, 28, 28))
    
    # Calculate the accuracy for each frame in the movie
    mismatch_found = False
    for frame_idx in range(20):
        frame_acc = (np.argmax(predicted_labels[0][frame_idx]) == (movie_labels[frame_idx]))
        # eliminate the first frame 
        frame_accs[frame_idx] += (frame_acc)
        
        # Display the digit image and its predicted label

        # Repeat the label for each frame in the movie
        label = movie_labels[frame_idx]
        # Print the accuracy for this frame
        # print(f"Movie {movie_idx+1}, Frame {frame_idx+1} Accuracy: {(frame_acc)}")
        print(f"Movie {movie_idx+1}, Frame {frame_idx+1} Accuracy: {(frame_acc)},predicted_label:{np.argmax(predicted_labels[0][frame_idx])}, actual label:{(movie_labels[frame_idx])}")
        # digit = movie_data[frame_idx, :, :]
        # digit_label = np.argmax(predicted_labels[0][frame_idx])
        # digit = digit.reshape(28, 28)
        # plt.imshow(digit)
        # plt.title(f"Frame {frame_idx+1}, Digit Label: {digit_label}")
        # plt.show()

    # if mismatch_found:
    #    mismatched_movies.append(movie_idx)
    # Check if all frames match the labels

    all_correct = all(np.argmax(predicted_labels[0][i]) == (movie_labels[i]) for i in range(1, 20))
    if all_correct:
        num_movies_all_correct += 1

# Print the number of movies where all frames match the labels
print(f"Number of movies where all frames match the labels: {num_movies_all_correct[0]}")
    
# Divide by the number of movies to get the mean accuracy for each frame
frame_accs /= len(x_test_mask_movie_sq)

# Print the mean accuracy for each frame
# for frame_idx in range(20):
#     print(f"Frame {frame_idx+1} Accuracy: {frame_accs[frame_idx]}")

In [ ]:
n1=10
dict_hist = history3.history # assuming you have already trained your model and stored the history object

list_ep = [i for i in range(1,n1+1)]

plt.figure(figsize=(8,8))

plt.subplot(2, 1, 1) # adding a subplot for accuracy
plt.plot(list_ep, dict_hist['accuracy'],ls='--',label='accuracy-train_dataset')
plt.plot(list_ep, dict_hist['val_accuracy'],ls='--',label='accuracy-validation_dataset')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 1, 2) # adding a subplot for loss
plt.plot(list_ep, dict_hist['loss'],ls='--',label='loss-train_dataset')
plt.plot(list_ep, dict_hist['val_loss'],ls='--',label='loss-validation_dataset')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()

plt.show()

### **Visualize Predicted Labels**

In [ ]:
# Choose a movie index to plot its frames
movie_idx =9198

# Get the data and labels for the chosen movie
movie_data = x_test_mask_movie_sq[movie_idx]
movie_labels = y_test[movie_idx]

# Repeat the labels for each frame in the movie ( the first frame is eliminated )
frame_num = 20
movie_labels = np.repeat(movie_labels, frame_num, axis=0)
movie_labels = movie_labels.reshape(frame_num, -1)

# Predict the labels for each frame in the movie
predicted_labels = model3.predict(movie_data.reshape(1, 20, 28, 28))




fig, axs = plt.subplots(1, num_frames, figsize=(100, 30), subplot_kw={'aspect': 'equal'})
# loop through the frames and display each one separately
for i in range(num_frames):
    digit = movie_data[i, :, :]
    digit_label = np.argmax(predicted_labels[0][i])
    digit = digit.reshape(28, 28)
    axs[i].imshow(digit)
    axs[i].set_title(f"Movie {movie_idx+1}, Frame {i+1}, Digit Label: {digit_label}")
    axs[i].axis('off')
   


plt.show()
# # Loop through each frame in the movie and plot it with its predicted label
# for frame_idx in range(frame_num):
#     # Display the digit image and its predicted label
#     digit = movie_data[frame_idx, :, :]
#     digit_label = np.argmax(predicted_labels[0][frame_idx])
#     digit = digit.reshape(28, 28)
#     plt.imshow(digit)
#     plt.title(f"Movie {movie_idx+1}, Frame {frame_idx+1}, Digit Label: {digit_label}")
#     plt.show()

In [ ]:
### Input 
# the index of the sample you want to display
sample_index = 9198
# the number of frames in each image
num_frames = 20

fig, axs = plt.subplots(1, num_frames, figsize=(50, 30), subplot_kw={'aspect': 'equal'})

# loop through the frames and display each one separately
for i in range(num_frames):
    axs[i].imshow(x_test_mask_movie[sample_index, i, :, :])
    axs[i].set_title(f"Movie {sample_index + 1}, Frame {i + 1}")
    axs[i].axis('off')

plt.show()

# **Manually updating the weight values of PR layer**

In [ ]:
## get the weights for PR layer
import re
import os
# get all the weights for all the layers 
def get_weightsDict(mdl):
    names = [weight.name for layer in mdl.layers for weight in layer.weights]
    weights = mdl.get_weights()
    weights_dict = {}
    for i in range(len(names)):
        weight_name = names[i][:-2]
        weights_dict[weight_name] = np.atleast_1d(np.squeeze(weights[i]))
    return weights_dict, names 

# Get the weights for a specific layer 
def get_weightsOfLayer(weights_dict,layer_name):
    weights_keys = list(weights_dict.keys())
    rgb = re.compile(layer_name+'/')
    layer_weight_names = list(filter(rgb.match, weights_keys))
    weights_layer = {}
    for l_name in layer_weight_names:
        param_name_full = os.path.basename(l_name)
        rgb = re.findall(r'[^0-9]',param_name_full)
        rgb = ''.join(rgb)
        if rgb[-1] == '_':
            rgb = rgb[:-1]
        param_name = rgb
        weights_layer[param_name] = weights_dict[l_name]
    return weights_layer

In [ ]:
# tauc=0.039
# zeta_value=0.2
# kappa_value=0.13
# alpha_value=0.0667
# beta_value=0
# gamma=0.1
# tauy=0.06
# ny=0.1
# tauz=0.09
# nz=0.09
# nc=0.16

# new_weights=copy.deepcopy(old_weights)

# # tauC 
# new_weights[weight_name_to_index["tauC"]]=np.reshape((np.array([tauc],dtype=np.float32)),(1,1))
                                      
# # zeta 
# new_weights[weight_name_to_index["zeta"]]=np.reshape((np.array([zeta_value],dtype=np.float32)),(1,1))

# # zeta_multFac 
# # new_weights[2]=np.reshape((np.array([zeta_multFac_value],dtype=np.float32)),(1,1))

# # kappa 
# new_weights[weight_name_to_index["kappa"]]=np.reshape((np.array([kappa_value],dtype=np.float32)),(1,1))

# # kappa_multFac 
# # new_weights[4]=np.reshape((np.array([kappa_multFac],dtype=np.float32)),(1,1))

# # alpha 
# new_weights[weight_name_to_index["alpha"]]=np.reshape((np.array([alpha_value],dtype=np.float32)),(1,1))

# # alpha_multFac 
# # new_weights[6]=np.reshape((np.array([alpha_multFac],dtype=np.float32)),(1,1))

# # beta 
# new_weights[weight_name_to_index["beta"]]=np.reshape((np.array([beta_value],dtype=np.float32)),(1,1))

# # beta_multFac 
# # new_weights[8]=np.reshape((np.array([beta_multFac],dtype=np.float32)),(1,1))

# # gamma  
# new_weights[weight_name_to_index["gamma"]]=np.reshape((np.array([gamma],dtype=np.float32)),(1,1))

# # gamma_multFac 
# # new_weights[10]=np.reshape((np.array([gamma_multFac],dtype=np.float32)),(1,1))

# # tau_y 
# new_weights[weight_name_to_index["tauY"]]=np.reshape((np.array([tauy],dtype=np.float32)),(1,1))

# # # tau_y_multFac
# # new_weights[12]=np.reshape((np.array([tau_y_multFac],dtype=np.float32)),(1,1))

# # # n_y 
# new_weights[weight_name_to_index["nY"]]=np.reshape((np.array([ny],dtype=np.float32)),(1,1))
                                      
# # # # n_y_multFac 
# # new_weights[13]=np.reshape((np.array([n_y_multFac],dtype=np.float32)),(1,1))

# # # tauZ 
# new_weights[weight_name_to_index["tauZ"]]=np.reshape((np.array([tauz],dtype=np.float32)),(1,1))
          
# # # tauZ_multFac 
# # new_weights[16]=np.reshape((np.array([tau_z_multFac],dtype=np.float32)),(1,1))

# # # n_z 
# new_weights[weight_name_to_index["nZ"]]=np.reshape((np.array([nz],dtype=np.float32)),(1,1))

# # # n_z_multFac 
# # new_weights[18]=np.reshape((np.array([n_z_multFac],dtype=np.float32)),(1,1))

# # # tauC_multFac 
# # new_weights[19]=np.reshape((np.array([tau_c_multFac],dtype=np.float32)),(1,1))                             

# # # nc 
# new_weights[weight_name_to_index["nC"]]=np.reshape((np.array([nc],dtype=np.float32)),(1,1)) 

# # # nc_multFac 
# # new_weights[19]=np.reshape((np.array([n_c_multFac],dtype=np.float32)),(1,1)) 

# ######### update the model here 
# submodel_PR.layers[2].set_weights(new_weights)

# output=submodel_PR.predict(x_test_mask_movie_sq)

# sample_index = 100  # Choose the index of the sample you want to visualize

# plt.figure(figsize=(5, 5))
# plt.imshow(output[sample_index, :, :])
# plt.title(f"Output of the PR model - Sample {sample_index}")
# plt.colorbar()
# plt.show()

In [ ]:
# Weight values of PR Layer
weights, names=get_weightsDict(model2)
s=names[1]
substring = s.split("/")[0]
layer_name=substring
W_L=get_weightsOfLayer(weights,layer_name)
print(W_L)
weights = submodel_PR2.layers[2].get_weights()
# print(weights)

# **Optimal Values for PR Parameters**

In [ ]:
for i in range(len(weights)):
    print(f"Weight {i} name: {submodel_PR2.layers[2].weights[i].name}")

In [ ]:
# create a dictionary that maps weight names to their indices 
weight_name_to_index={}

weights = submodel_PR2.layers[2].get_weights()
for i in range(len(weights)):
    print(f"Weight {i} name: {submodel_PR2.layers[2].weights[i].name}")

weight_names = []
for i, weight in enumerate(weights):
    weight_name = submodel_PR2.layers[2].weights[i].name
    weight_name = weight_name.split("/")[-1]  # Get the substring after the last slash
    weight_name = weight_name.split(":")[0]  # Get the substring before the colon
    weight_names.append(weight_name)

for i, weight_name in enumerate(weight_names):
  weight_name_to_index[weight_name]=i


In [ ]:
def compute_rms_error(batch_image1,batch_image2):
    assert batch_image1.shape[:1] == batch_image2.shape[:1], "Input batches must have the same number of samples"
    # use the first frame 
    first_frames_image1 = batch_image1[:, 0]
    error = np.square(first_frames_image1.astype(np.float64) - batch_image2.astype(np.float64))
    rms_errors = np.sqrt(np.mean(error, axis=(1, 2)))

    return rms_errors

In [ ]:
# Kz
# import os
# from concurrent.futures import ProcessPoolExecutor, as_completed,ThreadPoolExecutor
# import numpy as np
# import copy



# # ranges of PR model parameters ( firstset )
# # tauy_range= np.linspace(0.01, 0.06, 3)
# # ny_range = np.linspace(0.00001, 0.1, 3)
# # gamma_range=np.linspace(0.01, 0.1, 3)
# # tauz_range= np.linspace(0.05, 0.09, 3)
# # nz_range = np.linspace(0.01, 0.09, 3)

# # # optimal values 
# # tauy=0.06
# # gamma=0.1
# # ny=0.1
# # tauz=0.09
# # nz=0.09
# # tauc=0.01
# # nc=0.05


# # # optimal values 
# tauy=0.0199
# gamma=0.1
# ny=0.06
# tauz=0.05
# nz=0.06
# tauc=0.01
# nc=0.05


# # optimal values 
# # tauy=0.06
# # gamma=0.1
# # ny=0.1
# # tauz=0.09
# # nz=0.09
# # tauc=0.01
# # nc=0.05





# # ranges of PR model parameters ( secondset )
# zeta_range=np.linspace(0,0.2,4)
# kappa_range=np.linspace(0,0.2,4)
# alpha_range=np.linspace(0,0.2,4)
# beta_range=np.linspace(0,0.2,4)

# # optimal valuess
# # zeta=0
# # kappa=0.005
# # alpha=0.1
# # beta=0.001




# # create a matrix to store the RMS error (5D array)
# # rms_errors = np.zeros((len(tauy_range), len(ny_range), len(gamma_range), len(tauz_range), len(nz_range), x_test_mask_movie_sq.shape[0]))
# rms_errors = np.zeros((len(zeta_range),len(kappa_range),len(alpha_range),len(beta_range), x_test_mask_movie_sq.shape[0]))




# # Loop through the range of parameters 


# # for i, tau_y in enumerate(tauy_range):
# #     for j, n_y in enumerate(ny_range):
# #         for k, gamma_value in enumerate(gamma_range):
# #             for l, tau_z in enumerate(tauz_range):
# #                 for m, n_z in enumerate(nz_range):

# for p, zeta_value in enumerate(zeta_range):
#     for q, kappa_value in enumerate(kappa_range):
#         for r, alpha_value in enumerate (alpha_range):
#             for s, beta_value in enumerate (beta_range):

# ###### update the model here 

#                         old_weights=submodel_PR2.layers[2].get_weights()
#                         new_weights=copy.deepcopy(old_weights)

#                         # tauC 
#                         new_weights[weight_name_to_index["tauC"]]=np.reshape((np.array([tauc],dtype=np.float32)),(1,1))
                                      
#                         # zeta 
#                         new_weights[weight_name_to_index["zeta"]]=np.reshape((np.array([zeta_value],dtype=np.float32)),(1,1))

#                       # zeta_multFac 
#                         # new_weights[2]=np.reshape((np.array([zeta_multFac_value],dtype=np.float32)),(1,1))

#                       # kappa 
#                         new_weights[weight_name_to_index["kappa"]]=np.reshape((np.array([kappa_value],dtype=np.float32)),(1,1))

#                       # kappa_multFac 
#                         # new_weights[4]=np.reshape((np.array([kappa_multFac],dtype=np.float32)),(1,1))

#                       # alpha 
#                         new_weights[weight_name_to_index["alpha"]]=np.reshape((np.array([alpha_value],dtype=np.float32)),(1,1))

#                         # alpha_multFac 
#                       # new_weights[6]=np.reshape((np.array([alpha_multFac],dtype=np.float32)),(1,1))

#                       # beta 
#                         new_weights[weight_name_to_index["beta"]]=np.reshape((np.array([beta_value],dtype=np.float32)),(1,1))

#                     # beta_multFac 
#                       # new_weights[8]=np.reshape((np.array([beta_multFac],dtype=np.float32)),(1,1))

#                       # gamma  
#                         new_weights[weight_name_to_index["gamma"]]=np.reshape((np.array([gamma],dtype=np.float32)),(1,1))

#                   # gamma_multFac 
#                     # new_weights[10]=np.reshape((np.array([gamma_multFac],dtype=np.float32)),(1,1))

#                       # tau_y 
#                         new_weights[weight_name_to_index["tauY"]]=np.reshape((np.array([tauy],dtype=np.float32)),(1,1))

#                   # # tau_y_multFac
#                       # new_weights[12]=np.reshape((np.array([tau_y_multFac],dtype=np.float32)),(1,1))

#                       # # n_y 
#                         new_weights[weight_name_to_index["nY"]]=np.reshape((np.array([ny],dtype=np.float32)),(1,1))
                                      
#                         # # # n_y_multFac 
#                       # new_weights[13]=np.reshape((np.array([n_y_multFac],dtype=np.float32)),(1,1))

#                       # # tauZ 
#                         new_weights[weight_name_to_index["tauZ"]]=np.reshape((np.array([tauz],dtype=np.float32)),(1,1))
          
#                       # # tauZ_multFac 
#                         # new_weights[16]=np.reshape((np.array([tau_z_multFac],dtype=np.float32)),(1,1))

#                         # # n_z 
#                         new_weights[weight_name_to_index["nZ"]]=np.reshape((np.array([nz],dtype=np.float32)),(1,1))

#                       # # n_z_multFac 
#                       # new_weights[18]=np.reshape((np.array([n_z_multFac],dtype=np.float32)),(1,1))

#                         # # tauC_multFac 
#                       # new_weights[19]=np.reshape((np.array([tau_c_multFac],dtype=np.float32)),(1,1))                             

#                       # # nc 
#                         new_weights[weight_name_to_index["nC"]]=np.reshape((np.array([nc],dtype=np.float32)),(1,1)) 

#                         # # nc_multFac 
#                       # new_weights[19]=np.reshape((np.array([n_c_multFac],dtype=np.float32)),(1,1)) 

#     ######### update the model here 
#                         submodel_PR2.layers[2].set_weights(new_weights)

#                         output=submodel_PR2.predict(x_test_mask_movie_sq)

                                      

#                         # get the output of photoreceptor 
#                         sample_index=100
#                         plt.figure(figsize=(5, 5))
#                         # onlu check the last frame 
#                         plt.imshow(output[sample_index,2, :, :])
#                         # plt.title(f"tau_y={tauy_range[i]:.3f}, n_y={ny_range[j]:.3f}, gamma={gamma_range[k]:.3f}, tau_z={tauz_range[l]:.3f}, n_z={nz_range[m]:.3f}", fontsize=8)
#                         plt.title(f"zeta={zeta_range[p]:.3f},kappa={kappa_range[q]:.4f},alpha={alpha_range[r]:.4f},beta={beta_range[s]:.4f}", fontsize=8)

#                         # plt.colorbar()
#                         # plt.show()
#                         # save animation as an MP4 video file
#                         # filename=f"tau_y={tauy_range[i]:.3f}_n_y={ny_range[j]:.3f}_gamma={gamma_range[k]:.3f}_tau_z={tauz_range[l]:.3f}_n_z={nz_range[m]:.3f}.png"
#                         filename=f"zeta={zeta_range[p]:.3f}_kappa={kappa_range[q]:.3f}_alpha={alpha_range[r]:.3f}_beta={beta_range[s]:.3f}.png"
#                         digits_path = '/content/drive/MyDrive/PhD-Nilou/Project#3_DynamicCoding/Nilou_Codes/Tests/Test_25/'
#                       # Combine the digits_path with the filename
#                         output_path = os.path.join(digits_path, filename)
#                         plt.savefig(output_path, dpi=300)
#                         plt.close()

#                         # extract first frame of the onput
#                         # first_frame_input=x_test_mask_movie_sq[0]

#                       # compute RMS between the input and output 
#                         # Remove the last dimension of the output if it is (10000, 28, 28, 1)

#                         # output = np.squeeze(output, axis=-1)

#                         # rms_error=compute_rms_error(x_test_mask_movie_sq,output)
#                         # # rms_errors[i, j, k, l, m,:] = rms_error
#                         # rms_errors[p, q, r, s,:] = rms_error
                              

In [ ]:
# # Find the min RMS value for each combination 
# min_rms_value = np.min(rms_errors)
# print(min_rms_value)
# min_rms_index = np.argmin(rms_errors)
# print(min_rms_index)
# min_rms_multi_index = np.unravel_index(min_rms_index, rms_errors.shape)
# print(min_rms_multi_index)

# # min_tau_y=min_rms_multi_index[0]
# # min_n_y=min_rms_multi_index[1]
# # min_gamma=min_rms_multi_index[2]
# # min_tau_z=min_rms_multi_index[3]
# # min_n_z=min_rms_multi_index[4]

# min_zeta=min_rms_multi_index[0]
# min_kappa=min_rms_multi_index[1]
# min_alpha=min_rms_multi_index[2]
# min_beta=min_rms_multi_index[3]

# # min_tau_y = tauy_range[min_rms_multi_index[0]]
# # min_n_y = ny_range[min_rms_multi_index[1]]
# # min_gamma = gamma_range[min_rms_multi_index[2]]
# # min_tau_z = tauz_range[min_rms_multi_index[3]]
# # min_n_z = nz_range[min_rms_multi_index[4]]

# min_zeta=zeta_range[min_rms_multi_index[0]]
# min_kappa=kappa_range[min_rms_multi_index[1]]
# min_alpha=alpha_range[min_rms_multi_index[2]]
# min_beta=beta_range[min_rms_multi_index[3]]

# print(f"Minimum RMS found at:")

# # print(f"tau_y: {min_tau_y}")
# # print(f"n_y: {min_n_y}")
# # print(f"gamma: {min_gamma}")
# # print(f"tau_z: {min_tau_z}")
# # print(f"n_z: {min_n_z}")

# print(f"zeta: {min_zeta}")
# print(f"kappa: {min_kappa}")
# print(f"alpha: {min_alpha}")
# print(f"beta: {min_beta}")

# **Ky and Kz Linear Filters**

In [ ]:
# Get the filters 
def generate_simple_filter(tau,n,t):
    t_rep = np.repeat(t[:,None],n.shape[0],axis=-1)
    f = (t_rep**n[None,:])*np.exp(-t_rep/tau[None,:]); # functional form in paper
    f = (f/tau**(n+1))/scipy_gamma(n+1) # normalize appropriately
        # f = f/np.sum(f,axis=0)
    return f

In [ ]:
def generate_simple_filter_multichan(tau,n,t):
    t_shape = t.shape[0]
    t = tf.tile(t,tf.constant([tau.shape[-1]], tf.int32))
    t = tf.reshape(t,(tau.shape[-1],t_shape))
    t = tf.transpose(t)
    f = (t**n[:,None])*tf.math.exp(-t/tau[:,None]) # functional form in paper
    rgb = tau**(n+1)
    f = (f/rgb[:,None])/tf.math.exp(tf.math.lgamma(n+1))[:,None] # normalize appropriately
    # print(t.shape)
    # print(n.shape)
    # print(tau.shape)
   
    return f

## **Filters For All the Channels**

In [ ]:
# Ky filter for each channel ( there are 20 channels )
t = tf.range(0,inputs.shape[1],dtype='float32')


# get the weight pf PR layer 
tau_values=W_L['tauY']*W_L['tauY_mulFac']
n_values=W_L['nY']*W_L['nY_mulFac']


for i in range(len(tau_values)):
    tau = tau_values[i]
    n = n_values[i]

    # Generate filters using the current tau and n values
    filters_random = generate_simple_filter_multichan(np.array([tau], dtype=np.float32), np.array([n], dtype=np.float32), t)

    # Plot the filter with a label
    plt.plot(filters_random[:, 0], label=f"tauY = {tau:.4f}, nY = {n:.4f}, channel={i+1}")

# Set the title, x and y axis labels
plt.title("Ky Linear Filter", fontsize=30)
plt.xlabel("Frame", fontsize=20)
plt.ylabel("Gain", fontsize=20)

# Get the current x-axis tick locations and round them to the nearest integer
# Create an array of evenly spaced values from 0 to 20
xticks = np.linspace(0, 20, 21)

# Set the x-axis tick locations to the created array
plt.xticks(xticks, xticks.astype(int))

# Add a legend
plt.legend(fontsize=12, loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()

In [ ]:
# Kz filter for each channel ( there are 20 channels )
t = tf.range(0,inputs.shape[1],dtype='float32')


# get the weight pf PR layer 
tauY_values=W_L['tauY']*W_L['tauY_mulFac']
nY_values=W_L['nY']*W_L['nY_mulFac']
nZ_values=W_L['nZ']*W_L['nZ_mulFac']
tauZ_values=W_L['tauZ']*W_L['tauZ_mulFac']
gamma_values=W_L['gamma']*W_L['gamma_mulFac']

for i in range(len(tauY_values)):
    tauY = tauY_values[i]
    nY = nY_values[i]
    tauZ=tauZ_values[i]
    nZ=nZ_values[i]
    gamma=gamma_values[i]

        # Generate filters using the current tau and n values
    ky =generate_simple_filter_multichan(np.array([tauY], dtype=np.float32), np.array([nY], dtype=np.float32), t)
    kz=generate_simple_filter_multichan(np.array([tauZ], dtype=np.float32), np.array([nZ], dtype=np.float32), t)
    kz=(gamma*ky)+(1-gamma)*kz

    # Plot the filter with a label
    plt.plot(kz[:, 0], label=f"tauY = {tauY:.4f}, nY = {nY:.4f}, tauZ={tauZ:.4f},nZ={nZ:.4f}, gamma={gamma:.4f}, channel={i+1}")

# Set the title, x and y axis labels
plt.title("Kz Linear Filter", fontsize=30)
plt.xlabel("Frame", fontsize=20)
plt.ylabel("Gain", fontsize=20)
# Get the current x-axis tick locations and round them to the nearest integer
# Create an array of evenly spaced values from 0 to 20
xticks = np.linspace(0, 20, 21)

# Set the x-axis tick locations to the created array
plt.xticks(xticks, xticks.astype(int))

# Add a legend
plt.legend(fontsize=12, loc='center left', bbox_to_anchor=(1, 0.5))

# Add a legend
plt.legend(fontsize=12, loc='center left', bbox_to_anchor=(1, 0.5))

# Show the plot
plt.show()